# Рассчитываем хорошие и плохие вектора и близость до них

Цель: предсказать для каждого пользователя взятие/ невзятие каждого из четырех продуктов **в течение месяца после отчетной даты**, исторические данные по ним находятся в targets

In [ ]:
import numpy as np

import pandas as pd
from pandas.api.types import is_float_dtype, is_integer_dtype

from collections import Counter
from sklearn.utils import resample

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import math

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

import gc
import glob
import pyarrow.parquet as pq
from tqdm import trange, tqdm

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform

In [ ]:
from typing import List, Optional, Tuple

In [ ]:
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# найтройки
# Убираем ограничение отображемых колонок
pd.set_option("display.max_columns", None)
# Устанавливаем тему по умолчанию
sb_dark = sns.dark_palette('skyblue', 8, reverse=True) # teal
sns.set(palette=sb_dark)

In [ ]:
# Включаем tqdm для pandas, чтобы можно было запускать progress_apply() вместо простого apply()
tqdm.pandas() 
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

In [ ]:
eps = 1e-6

In [ ]:
PATH = ''
PATH_DATASET = PATH + 'datasets/sber_source/'
PATH_DATASET_OUTPUT = PATH + 'datasets/'

PATH_DATASET_DIALOG_TRAIN = PATH_DATASET + 'dial_train.parquet/'
PATH_DATASET_DIALOG_TEST = PATH_DATASET + 'dial_test.parquet/'

PATH_DATASET_TARGET_TRAIN = PATH_DATASET + 'train_target.parquet/'
PATH_DATASET_TARGET_TEST = PATH_DATASET + 'test_target_b.parquet/'


# dialogs
train_dialog_files = glob.glob(PATH_DATASET_DIALOG_TRAIN + '/*.parquet')
test_dialog_files = glob.glob(PATH_DATASET_DIALOG_TEST + '/*.parquet')

# Файлы таргеты
train_target_files = glob.glob(PATH_DATASET_TARGET_TRAIN + '/*.parquet')
test_target_files = glob.glob(PATH_DATASET_TARGET_TEST + '/*.parquet')


len(train_dialog_files), len(test_dialog_files)

In [ ]:
# Загрузка списка файлов (типа паркет) в один датафрейм
def load_df_by_files(files:list[str]) -> pd.DataFrame:
    union_df = pd.DataFrame()
    for file in tqdm(files):
        current_df = pq.read_table(file).to_pandas()    
        union_df = pd.concat([union_df, current_df])
    return union_df

In [ ]:
%%time
# Загружаем все таргеты
all_target_df = load_df_by_files(train_target_files + test_target_files)
all_target_df.shape

# Диалоги Рассчитываем хорошие и плохие вектора и близость до них

In [ ]:
emb_dial_select_1_df = pq.read_table(PATH_DATASET_OUTPUT + 'embeddings/' + 'emb_dial_v6.1_K-Means.parquet').to_pandas()
emb_dial_select_1_df.shape

In [ ]:
emb_dial_select_1_df = emb_dial_select_1_df.rename(columns={'emb_trx': 'embedding'})
emb_dial_select_1_df = emb_dial_select_1_df.drop(columns=['year', 'event_time', 'mon', 'target_1', 'target_2', 'target_3', 'target_4'], errors='ignore')
emb_dial_select_1_df.shape

In [ ]:
emb_dial_select_1_df['report_next_end'] = pd.to_datetime(emb_dial_select_1_df['report_next_end'])
emb_dial_select_1_df = emb_dial_select_1_df.set_index(['client_id', 'report_next_end'])
emb_dial_select_1_df.shape

# Рассчитываем хорошие и плохие диалоги

In [ ]:
all_target_df['mon'] = pd.to_datetime(all_target_df['mon'])
all_target_df = all_target_df.rename(columns={'mon': 'report_next_end'})
all_target_df = all_target_df.set_index(['client_id', 'report_next_end'])
all_target_df['is_target'] = all_target_df[['target_1', 'target_2', 'target_3', 'target_4']].max(axis=1)
all_target_df.shape

In [ ]:
emb_dial_select_1_df = all_target_df.merge(emb_dial_select_1_df, left_index=True, right_index=True)
emb_dial_select_1_df.shape

In [ ]:
emb_dial_select_1_df = emb_dial_select_1_df.drop(columns=['target_1', 'target_2', 'target_3', 'target_4', 'is_target', 'embedding'], errors='ignore')
emb_dial_select_1_df.shape

In [ ]:
emb_dial_select_1_df = emb_dial_select_1_df.add_prefix('geo_emb_v2__')
emb_dial_select_1_df.shape

In [ ]:
%%time
# Сохраняем статистику по косинусной близости
emb_dial_select_1_df.to_parquet(PATH_DATASET_OUTPUT + 'good_bad_dialog_target_df_v2_16_06_2024.parquet')
emb_dial_select_1_df.shape

# ГЕО Рассчитываем хорошие и плохие вектора и близость до них

In [ ]:
geo_emb_select_1_df = pq.read_table(PATH_DATASET_OUTPUT + 'embeddings/' + 'geo_emb_select_1_v3_K-Means.parquet').to_pandas()
geo_emb_select_1_df.shape

In [ ]:
geo_emb_select_1_df = geo_emb_select_1_df.rename(columns={'emb_trx': 'embedding'})
geo_emb_select_1_df = geo_emb_select_1_df.drop(columns=['year', 'event_time', 'mon', 'target_1', 'target_2', 'target_3', 'target_4'], errors='ignore')
geo_emb_select_1_df.shape

In [ ]:
geo_emb_select_1_df['report_next_end'] = pd.to_datetime(geo_emb_select_1_df['report_next_end'])
geo_emb_select_1_df = geo_emb_select_1_df.set_index(['client_id', 'report_next_end'])
geo_emb_select_1_df.shape

# Рассчитываем хорошие и плохие диалоги

In [ ]:
all_target_df['mon'] = pd.to_datetime(all_target_df['mon'])
all_target_df = all_target_df.rename(columns={'mon': 'report_next_end'})
all_target_df = all_target_df.set_index(['client_id', 'report_next_end'])
all_target_df['is_target'] = all_target_df[['target_1', 'target_2', 'target_3', 'target_4']].max(axis=1)
all_target_df.shape

In [ ]:
geo_emb_select_1_df = all_target_df.merge(geo_emb_select_1_df, left_index=True, right_index=True)
geo_emb_select_1_df.shape

In [ ]:
%%time
# Считаем "хорошие" и "плохие" эмбеддинги
target_columns = ['target_1', 'target_2', 'target_3', 'target_4', 'is_target']

target_embeddings_dict ={}
dialog_embeddings = np.vstack(geo_emb_select_1_df['embedding'].apply(np.array).values)

for trg_col in target_columns:
    print(trg_col)
    
    good_sub_dlg_trg_df = geo_emb_select_1_df[geo_emb_select_1_df[trg_col] == 1]
    bad_sub_dlg_trg_df = geo_emb_select_1_df[geo_emb_select_1_df[trg_col] == 0]
    good_embedding = good_sub_dlg_trg_df['embedding'].mean()
    bad_embedding = bad_sub_dlg_trg_df['embedding'].mean()
    
    target_embeddings_dict[trg_col] = {}
    target_embeddings_dict[trg_col]['good'] = good_embedding
    target_embeddings_dict[trg_col]['bad'] = bad_embedding
    
    # Производим расчет косинусного расстояния
    geo_emb_select_1_df[f'cos_sim_{trg_col}_good'] = cosine_similarity(dialog_embeddings, good_embedding[None])
    geo_emb_select_1_df[f'cos_sim_{trg_col}_bad'] = cosine_similarity(dialog_embeddings, bad_embedding[None])

In [ ]:
# Расчет статистик на данных
good_columns = []
bad_columns = []
for trg_col in target_columns:
    print(trg_col)
    good_col = f'cos_sim_{trg_col}_good'
    bad_col = f'cos_sim_{trg_col}_bad'
    good_columns.append(good_col)
    bad_columns.append(bad_col)
    
    geo_emb_select_1_df['dist_bad_good'] = abs(geo_emb_select_1_df[bad_col] - geo_emb_select_1_df[good_col])
    geo_emb_select_1_df['diff_bad_good'] = geo_emb_select_1_df[bad_col] - geo_emb_select_1_df[good_col]
    geo_emb_select_1_df['rel_bad_good'] = geo_emb_select_1_df[bad_col] / (geo_emb_select_1_df[good_col] + eps)

In [ ]:
for t in ['bad', 'good']:
    geo_emb_select_1_df[f'mean_{t}_cossim'] = geo_emb_select_1_df[bad_columns].mean(axis=1)
    geo_emb_select_1_df[f'max_{t}_cossim'] = geo_emb_select_1_df[bad_columns].max(axis=1)
    geo_emb_select_1_df[f'min_{t}_cossim'] = geo_emb_select_1_df[bad_columns].min(axis=1)
    geo_emb_select_1_df[f'sum_{t}_cossim'] = geo_emb_select_1_df[bad_columns].sum(axis=1)

In [ ]:
geo_emb_select_1_df = geo_emb_select_1_df.drop(columns=['target_1', 'target_2', 'target_3', 'target_4', 'is_target', 'embedding'])
geo_emb_select_1_df.shape

In [ ]:
geo_emb_select_1_df = geo_emb_select_1_df.add_prefix('geo_emb__')
geo_emb_select_1_df.shape

In [ ]:
# %%time
# # Сохраняем статистику по косинусной близости
# geo_emb_select_1_df.to_parquet(PATH_DATASET_OUTPUT + 'good_bad_geo_target_df_16_06_2024.parquet')
# geo_emb_select_1_df.shape

# Транзакции Рассчитываем хорошие и плохие вектора и близость до них

In [ ]:
trx_emb_select_1_df = pq.read_table(PATH_DATASET_OUTPUT + 'embeddings/' + 'trx_emb_select_1_v3_K-Means.parquet').to_pandas()
trx_emb_select_1_df.shape

In [ ]:
trx_emb_select_1_df

In [ ]:
trx_emb_select_1_df = trx_emb_select_1_df.rename(columns={'emb_trx': 'embedding'})
trx_emb_select_1_df = trx_emb_select_1_df.drop(columns=['year', 'event_time', 'mon', 'target_1', 'target_2', 'target_3', 'target_4'], errors='ignore')
trx_emb_select_1_df.shape

In [ ]:
trx_emb_select_1_df['report_next_end'] = pd.to_datetime(trx_emb_select_1_df['report_next_end'])
trx_emb_select_1_df = trx_emb_select_1_df.set_index(['client_id', 'report_next_end'])
trx_emb_select_1_df.shape

# Рассчитываем хорошие и плохие диалоги

In [ ]:
all_target_df['mon'] = pd.to_datetime(all_target_df['mon'])
all_target_df = all_target_df.rename(columns={'mon': 'report_next_end'})
all_target_df = all_target_df.set_index(['client_id', 'report_next_end'])
all_target_df['is_target'] = all_target_df[['target_1', 'target_2', 'target_3', 'target_4']].max(axis=1)
all_target_df

In [ ]:
trx_emb_select_1_df = all_target_df.merge(trx_emb_select_1_df, left_index=True, right_index=True)
trx_emb_select_1_df.shape

In [ ]:
%%time
# Считаем "хорошие" и "плохие" эмбеддинги
target_columns = ['target_1', 'target_2', 'target_3', 'target_4', 'is_target']

target_embeddings_dict ={}
dialog_embeddings = np.vstack(trx_emb_select_1_df['embedding'].apply(np.array).values)

for trg_col in target_columns:
    print(trg_col)
    
    good_sub_dlg_trg_df = trx_emb_select_1_df[trx_emb_select_1_df[trg_col] == 1]
    bad_sub_dlg_trg_df = trx_emb_select_1_df[trx_emb_select_1_df[trg_col] == 0]
    good_embedding = good_sub_dlg_trg_df['embedding'].mean()
    bad_embedding = bad_sub_dlg_trg_df['embedding'].mean()
    
    target_embeddings_dict[trg_col] = {}
    target_embeddings_dict[trg_col]['good'] = good_embedding
    target_embeddings_dict[trg_col]['bad'] = bad_embedding
    
    # Производим расчет косинусного расстояния
    trx_emb_select_1_df[f'cos_sim_{trg_col}_good'] = cosine_similarity(dialog_embeddings, good_embedding[None])
    trx_emb_select_1_df[f'cos_sim_{trg_col}_bad'] = cosine_similarity(dialog_embeddings, bad_embedding[None])

In [ ]:
# Расчет статистик на данных
good_columns = []
bad_columns = []
for trg_col in target_columns:
    print(trg_col)
    good_col = f'cos_sim_{trg_col}_good'
    bad_col = f'cos_sim_{trg_col}_bad'
    good_columns.append(good_col)
    bad_columns.append(bad_col)
    
    trx_emb_select_1_df['dist_bad_good'] = abs(trx_emb_select_1_df[bad_col] - trx_emb_select_1_df[good_col])
    trx_emb_select_1_df['diff_bad_good'] = trx_emb_select_1_df[bad_col] - trx_emb_select_1_df[good_col]
    trx_emb_select_1_df['rel_bad_good'] = trx_emb_select_1_df[bad_col] / (trx_emb_select_1_df[good_col] + eps)

In [ ]:
for t in ['bad', 'good']:
    trx_emb_select_1_df[f'mean_{t}_cossim'] = trx_emb_select_1_df[bad_columns].mean(axis=1)
    trx_emb_select_1_df[f'max_{t}_cossim'] = trx_emb_select_1_df[bad_columns].max(axis=1)
    trx_emb_select_1_df[f'min_{t}_cossim'] = trx_emb_select_1_df[bad_columns].min(axis=1)
    trx_emb_select_1_df[f'sum_{t}_cossim'] = trx_emb_select_1_df[bad_columns].sum(axis=1)

In [ ]:
trx_emb_select_1_df = trx_emb_select_1_df.drop(columns=['target_1', 'target_2', 'target_3', 'target_4', 'is_target', 'embedding'])
trx_emb_select_1_df.shape

In [ ]:
trx_emb_select_1_df = trx_emb_select_1_df.add_prefix('trx_emb__')
trx_emb_select_1_df.shape

In [ ]:
trx_emb_select_1_df

# Диалоги Рассчитываем хорошие и плохие вектора и близость до них

In [ ]:
%%time
# Загружаем диалоги клиентов
# train_dialog_df = load_df_by_files(train_dialog_files)
# test_dialog_df = load_df_by_files(test_dialog_files)
# train_dialog_df.shape, test_dialog_df.shape
# all_dialogs_df = pd.concat([train_dialog_df, test_dialog_df])
all_dialogs_df = load_df_by_files(train_dialog_files + test_dialog_files)
all_dialogs_df.shape

In [ ]:
%%time
from pandas.tseries.offsets import MonthEnd
all_dialogs_df['end_month'] = pd.to_datetime(all_dialogs_df['event_time'], format="%Y%m") + MonthEnd(1)
all_dialogs_df['end_month'] = all_dialogs_df['end_month'].dt.date
all_dialogs_df['report_next_end'] = all_dialogs_df['end_month'] + relativedelta(months=1)
all_dialogs_df.shape

In [ ]:
# all_dialogs_df = all_dialogs_df[:1000]
all_dialogs_df = all_dialogs_df.drop(columns=['event_time', 'end_month'], errors='ignore')
all_dialogs_df['report_next_end'] = pd.to_datetime(all_dialogs_df['report_next_end'])
all_dialogs_df = all_dialogs_df.set_index(['client_id', 'report_next_end'])
all_dialogs_df.shape

In [ ]:
# # end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
# # report_next_end
# all_dialogs_df['end_month'] + relativedelta(months=1)

# Рассчитываем хорошие и плохие диалоги

In [ ]:
all_target_df['mon'] = pd.to_datetime(all_target_df['mon'])
all_target_df = all_target_df.rename(columns={'mon': 'report_next_end'})
all_target_df = all_target_df.set_index(['client_id', 'report_next_end'])
all_target_df['is_target'] = all_target_df[['target_1', 'target_2', 'target_3', 'target_4']].max(axis=1)
all_target_df

In [ ]:
dialogs_target_df = all_target_df.merge(all_dialogs_df, left_index=True, right_index=True)
dialogs_target_df.shape

In [ ]:
%%time
# Считаем "хорошие" и "плохие" эмбеддинги
target_columns = ['target_1', 'target_2', 'target_3', 'target_4', 'is_target']

target_embeddings_dict ={}
dialog_embeddings = np.vstack(dialogs_target_df['embedding'].apply(np.array).values)

for trg_col in target_columns:
    print(trg_col)
    
    good_sub_dlg_trg_df = dialogs_target_df[dialogs_target_df[trg_col] == 1]
    bad_sub_dlg_trg_df = dialogs_target_df[dialogs_target_df[trg_col] == 0]
    good_embedding = good_sub_dlg_trg_df['embedding'].mean()
    bad_embedding = bad_sub_dlg_trg_df['embedding'].mean()
    
    target_embeddings_dict[trg_col] = {}
    target_embeddings_dict[trg_col]['good'] = good_embedding
    target_embeddings_dict[trg_col]['bad'] = bad_embedding
    
    # Производим расчет косинусного расстояния
    dialogs_target_df[f'cos_sim_{trg_col}_good'] = cosine_similarity(dialog_embeddings, good_embedding[None])
    dialogs_target_df[f'cos_sim_{trg_col}_bad'] = cosine_similarity(dialog_embeddings, bad_embedding[None])

In [ ]:
# Расчет статистик на данных
good_columns = []
bad_columns = []
for trg_col in target_columns:
    print(trg_col)
    good_col = f'cos_sim_{trg_col}_good'
    bad_col = f'cos_sim_{trg_col}_bad'
    good_columns.append(good_col)
    bad_columns.append(bad_col)
    
    dialogs_target_df['dist_bad_good'] = abs(dialogs_target_df[bad_col] - dialogs_target_df[good_col])
    dialogs_target_df['diff_bad_good'] = dialogs_target_df[bad_col] - dialogs_target_df[good_col]
    dialogs_target_df['rel_bad_good'] = dialogs_target_df[bad_col] / (dialogs_target_df[good_col] + eps)

In [ ]:
for t in ['bad', 'good']:
    dialogs_target_df[f'mean_{t}_cossim'] = dialogs_target_df[bad_columns].mean(axis=1)
    dialogs_target_df[f'max_{t}_cossim'] = dialogs_target_df[bad_columns].max(axis=1)
    dialogs_target_df[f'min_{t}_cossim'] = dialogs_target_df[bad_columns].min(axis=1)
    dialogs_target_df[f'sum_{t}_cossim'] = dialogs_target_df[bad_columns].sum(axis=1)

In [ ]:
dialogs_target_df = dialogs_target_df.drop(columns=['target_1', 'target_2', 'target_3', 'target_4', 'is_target', 'embedding'])
dialogs_target_df.shape

In [ ]:
dialogs_target_df

In [ ]:
%%time
# Сохраняем статистику по косинусной близости
# dialogs_target_df.to_parquet(PATH_DATASET_OUTPUT + 'good_bad_dialogs_target_df_16_06_2024.parquet')

### Загружаем языковую модель

In [ ]:
%%time
bertmodel = SentenceTransformer('DeepPavlov/bart-base-en-persona-chat') # 768

## Формируем примеры диалогов по тематикам 
На базе https://claude.ai/  


In [ ]:
vocab_dialogs = {"Текущие (расчетные) счета": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, я бы хотел открыть расчетный счет в вашем банке для своего бизнеса. Оператор: Хорошо, для этого вам необходимо предоставить следующие документы: свидетельство о регистрации юридического лица, устав, приказ о назначении директора и другие учредительные документы. После их рассмотрения мы сможем открыть вам расчетный счет. Клиент: Отлично, большое спасибо за подробную информацию. Я соберу все необходимые документы и приду в ближайшее отделение."},
    {"Негативный диалог:": "Клиент: Я уже несколько раз обращался с просьбой открыть расчетный счет, но каждый раз мне говорят о новых требованиях к документам! Это издевательство! Оператор: Извините, но мы обязаны соблюдать требования законодательства и внутренних нормативов банка. Возможно, есть какие-то нюансы в вашей ситуации, и поэтому запрашиваются дополнительные документы. Давайте разберемся подробнее, что именно вам требуется?",},
],
"Депозиты (вклады)": [
    {"Позитивный диалог:": "Клиент: Добрый день, какие виды вкладов сейчас наиболее выгодные в вашем банке? Оператор: Здравствуйте. Сейчас наиболее привлекательными являются наши накопительные вклады со ставкой до 7% годовых. Вы можете открыть его с минимальной суммой от 50 000 рублей. Также есть возможность частичного снятия средств без потери процентов.Клиент: Отлично, тогда я бы хотел открыть такой вклад. Скажите, пожалуйста, какие документы нужно принести в отделение?"},
    {"Негативный диалог:": "Клиент: Я открыл вклад в вашем банке 3 месяца назад под 9% годовых. А сегодня вижу, что ставки снизились до 7%. Почему мне не предложили более выгодные условия?Оператор: К сожалению, условия по вкладам фиксируются на момент их открытия и действуют до окончания срока. Изменить ставку на более высокую в течение срока вклада невозможно.Клиент: Это некрасиво по отношению к постоянным клиентам банка! Вы должны ценить нас и предлагать лучшие условия!"},
],
"Кредиты": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, я недавно обращался за кредитом в ваш банк. Могу я узнать, на каком этапе рассмотрения моя заявка?Оператор: Да, конечно. Позвольте уточнить ваши персональные данные... Хорошо, ваша заявка уже одобрена. В ближайшие 2 дня с вами свяжется наш менеджер для дальнейшего оформления кредитной документации.Клиент: Отлично, большое спасибо за оперативность. Буду ждать звонка менеджера."},
    {"Негативный диалог:": "Клиент: Я крайне разочарован работой вашего банка! Мне отказали в кредите, хотя все требования были соблюдены. При этом никаких внятных объяснений не предоставили.Оператор: Сожалею, что у вас сложилось негативное впечатление. Решение об одобрении или отказе выносится на основании тщательного анализа кредитоспособности клиента и опирается на ряд факторов. Я проверю вашу ситуацию и дам более развернутый ответ."},
],
"Кредитные карты": [
    {"Позитивный диалог:": "Клиент: Добрый день, я бы хотел оформить кредитную карту. Какие условия сейчас действуют? Оператор: Здравствуйте. У нас сейчас очень выгодное предложение - кредитная карта с льготным периодом 100 дней без процентов на все покупки. Также есть начисление баллов за покупки, которые можно обменивать на подарки. Клиент: Отличное предложение! Скажите, какие документы нужно предоставить для оформления? Оператор: Для получения карты понадобятся паспорт, справка о доходах и анкета, которую можно заполнить у нас в отделении."},
    {"Негативный диалог:": "Клиент: Добрый день, моя кредитная карта вашего банка заблокирована уже несколько дней без объяснения причин! Требую разъяснить ситуацию! Оператор: Сожалею, ваша кредитная карта действительно была временно заблокирована нашей службой безопасности по причине подозрительной операции. Мы проводили дополнительную проверку, чтобы убедиться, что карта не была использована третьими лицами. Клиент: Это недопустимо! У меня были запланированы важные покупки, а из-за ваших необоснованных действий я понес убытки!"},
],
"Дебетовые карты": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, я хотел бы открыть вклад и оформить к нему дебетовую карту. Оператор: Хорошо, конечно. Вы можете выбрать подходящий вклад на нашем сайте в разделе 'Вклады и инвестиции'. Затем, открыв вклад, вы сможете сразу же оформить и получить дебетовую карту к нему для удобного управления средствами. Клиент: Отлично, спасибо за информацию. Я изучу предложения и открою вклад в ближайшее время."},
    {"Негативный диалог:": "Клиент: Добрый день! Моя зарплатная карта вашего банка была украдена, и с нее сняли все деньги. Почему средства не заблокированы своевременно? Оператор: Сожалею, что такая неприятная ситуация произошла. Для блокировки карты вам необходимо было незамедлительно обратиться в банк после кражи. Без вашего оповещения мы не могли знать о несанкционированных действиях с картой. Клиент: Но служба поддержки совершенно недоступна по вечерам и в выходные! Как клиент мог оперативно"},
],
"Денежные переводы и платежи (внутренние и международные)": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, мне нужно срочно перевести деньги родственникам за границу. Подскажите, как это можно сделать? Оператор: Здравствуйте. Для отправки денежного перевода за рубеж вам необходимо посетить любое отделение нашего банка и оформить международный перевод. Понадобится указать получателя, его реквизиты, сумму и валюту перевода. Комиссия будет зависеть от суммы и выбранной системы перевода. Клиент: Хорошо, спасибо большое за информацию. Тогда я приеду в ближайшее отделение и оформлю перевод."},
    {"Негативный диалог:": "Клиент: Я делал платеж через ваш интернет-банкинг на прошлой неделе, но деньги до сих пор не дошли получателю! Что за задержки? Оператор: Сожалею за доставленные неудобства. Для выяснения ситуации мне необходимо уточнить реквизиты платежа и способ его отправки. Переводы внутри страны обычно проходят в течение 1-2 рабочих дней. Клиент: Это просто безобразие! Я должен был оплатить важную услугу вовремя. Требую немедленного зачисления средств получателю и компенсации за задержку!"},
],
"Инвестиционные продукты (брокерские услуги, ПИФы, доверительное управление)": [
    {"Позитивный диалог:": "Клиент: Добрый день! Меня заинтересовали ваши инвестиционные услуги. Не могли бы вы рассказать подробнее о паевых инвестиционных фондах? Оператор: Конечно. ПИФы - это очень удобный инструмент для вложения средств под управлением профессиональной команды. У нас есть разные типы ПИФов: акций, облигаций, смешанных и других стратегий на выбор в зависимости от ваших инвестиционных целей и предпочтений по риску. Минимальная сумма покупки пая от 10 000 руб. Клиент: Хорошо, спасибо. Я изучу линейку ваших ПИФов и приму решение о покупке пая в ближайшее время."},
    {"Негативный диалог:": "Клиент: Я доверил вашим специалистам управление своим портфелем акций, а он принес убытки за год! Верну ли я свои вложенные средства? Оператор: Сожалею, что сложилась такая ситуация. Ввиду волатильности на рынках невозможно гарантировать прибыльность инвестиционного портфеля в краткосрочной перспективе. Но наши управляющие придерживаются разработанной стратегии, чтобы минимизировать риски. Клиент: Это неприемлемо! Я ожидал защиты моих средств, а не таких потерь. Прошу вернуть мои вложения в полном объеме!"},
],
"Банковские гарантии и аккредитивы": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, мне необходимо получить банковскую гарантию для обеспечения контракта. Каковы условия предоставления гарантий в вашем банке? Оператор: Для оформления гарантии нам потребуются учредительные документы вашей организации, финансовая отчетность, информация о контракте и обеспечении. После рассмотрения документов мы сможем назвать точные условия: сумму, сроки, тарифы. Клиент: Хорошо, большое спасибо. Я соберу пакет документов и приеду в ближайшее отделение для дальнейшего оформления."},
    {"Негативный диалог:": "Клиент: Вы уже месяц рассматриваете нашу заявку на получение аккредитива для расчетов с поставщиком. Сроки контракта горят! Почему так долго? Оператор: Извините за причиненные неудобства. Процедура рассмотрения аккредитивов более длительная, так как требует глубокого анализа контрактов и предполагаемых сделок на предмет экономических рисков. Мы стараемся максимально ускориться. Клиент: Это недопустимая волокита! Из-за ваших проволочек мы можем понести репутационные и финансовые потери. Прошу принять наконец положительное решение по нашему аккредитиву!"},
],
"Услуги инкассации и хранения ценностей": [
    {"Позитивный диалог:": "Клиент: Добрый день! Наша компания хотела бы воспользоваться услугами инкассации для перевозки наличных средств из торговых точек. Расскажите, пожалуйста, как мы можем это организовать? Оператор: Хорошо, для организации инкассаторских услуг вам необходимо заключить с нами договор. Сначала нужно предоставить пакет документов о компании, указать адреса забора и сумки. После этого мы составим маршрут и графики инкассации. Клиент: Отлично, большое спасибо за информацию. В ближайшее время наш специалист свяжется с вами для оформления договора."},
    {"Негативный диалог:": "Клиент: Добрый день! Я крайне возмущен работой вашей инкассаторской службы. Уже второй раз за месяц происходит задержка при вывозе наличных из наших магазинов! Оператор: Сожалею за доставленные неудобства. Такие ситуации недопустимы. Прошу прощения за сложившуюся ситуацию. Обязательно разберемся в причинах задержек и примем необходимые меры. Клиент: Это полное безобразие! Из-за подобных простоев мы несем финансовые потери. Прошу принять жесткие меры к исполнителям и гарантировать своевременное обслуживание по договору!"},
],
"Зарплатные проекты для организаций": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, меня интересует возможность перевода сотрудников нашей компании на зарплатный проект с вашим банком. Не могли бы вы рассказать об условиях и преимуществах? Оператор: Да, с удовольствием. Наш зарплатный проект предлагает ряд выгод как для вашей организации, так и для сотрудников. Для компании - это упрощение процедуры выплат зарплат, снижение издержек на обслуживание и специальные тарифы на РКО. Для сотрудников - бесплатное открытие и обслуживание карт, повышенный кэшбэк, льготные ставки по кредитам. Клиент: Звучит заманчиво. Расскажите, как можно подключиться к зарплатному проекту? Оператор: Для этого необходимо заключить договор зарплатного проекта с банком. Вам понадобится предоставить пакет учредительных документов компании, и мы рассмотрим заявку в короткие сроки."},
    {"Негативный диалог:": "Клиент: Я крайне недоволен вашим зарплатным обслуживанием! Уже третий месяц наши сотрудники жалуются на задержки зарплатных выплат. Это недопустимо! Оператор: Приношу свои извинения за доставленные неудобства. Это недопустимая ситуация, я обязательно разберусь в причинах задержек и передам информацию руководству для принятия мер. Клиент: Ваши извинения ничего не стоят! Из-за ваших постоянных сбоев мы рискуем потерять ценные кадры. Если ситуация не изменится, мы будем вынуждены отказаться от ваших услуг!"},
],
"Приложение банка": [
    {"Позитивный диалог:": "Клиент: Добрый день! Хотел узнать, есть ли у вашего банка мобильное приложение для управления счетами и картами? Оператор: Да, конечно. Мы разработали удобное мобильное приложение, которое позволяет полностью управлять вашими финансами удаленно. В приложении можно совершать переводы, погашать кредиты, пополнять вклады, анализировать расходы и многое другое. Клиент: Здорово! А как его можно установить? Оператор: Приложение доступно для скачивания в Play Market и App Store. После установки вам нужно будет пройти несложную регистрацию и авторизоваться, используя те же логин и пароль, что и в интернет-банке."},
    {"Негативный диалог:": "Клиент: Вашим мобильным приложением совершенно невозможно пользоваться - оно постоянно зависает и вылетает! Это ужасно! Оператор: Сожалею, что приложение доставляет вам неудобства. Мы стараемся постоянно совершенствовать его производительность и устранять замеченные недочеты. Не могли бы вы подробнее описать проблему? Это поможет разработчикам оперативно найти и исправить причину сбоев. Клиент: Да что тут описывать! Оно просто не работает нормально. То покупки не проходят, то платежи, то выдает ошибку при входе. Лучше полностью переделывайте приложение!"},
],
"Автокредиты": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, меня интересуют условия автокредитования в вашем банке. Я хотел бы приобрести новую машину в кредит. Оператор: Хорошо, для покупки нового автомобиля мы можем предложить вам выгодную программу автокредитования. Максимальный срок - 5 лет, ставка от 7,9% годовых, первоначальный взнос от 20%. Также предусмотрены программы с остаточным платежом и без первого взноса. Клиент: Замечательно, спасибо. А какие документы понадобятся для оформления кредита? Оператор: Для получения автокредита необходим паспорт, водительское удостоверение и справка о доходах. При визите в отделение наш менеджер сможет помочь собрать полный пакет документов."},
    {"Негативный диалог:": "Клиент: Я крайне возмущен работой вашего банка! Мне одобрили автокредит под гораздо более высокую ставку, чем та, что озвучивалась при рекламе вашего предложения. Оператор: Сожалею, что у вас сложилось негативное впечатление. В рекламных материалах мы указываем максимально низкие ставки, действующие только для наиболее надежных клиентов с идеальной кредитной историей. Конечретные условия определяются индивидуально после рассмотрения заявки. Клиент: Это недобросовестная реклама! Вы вводите клиентов в заблуждение. Отказываюсь от вашего кредита и буду жаловаться в надзорные органы!"},
],
"Кэшбэк": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, я пользуюсь вашей кредитной картой с начислением кэшбэка. Подскажите, как можно получить накопленный кэшбэк? Оператор: Приветствую! Для получения средств, накопленных по программе кэшбэка, вам нужно зайти в личный кабинет на сайте или в мобильном приложении. В разделе 'Бонусы и кэшбэк' будет указана доступная сумма для вывода на ваш счет или карту. Оформите заявку, и деньги будут зачислены в течение 3-5 рабочих дней. Клиент: Понятно, большое спасибо за разъяснение! Воспользуюсь личным кабинетом."},
    {"Негативный диалог:": "Клиент: Добрый день! Хочу высказать претензию по использованию вашей рекламной акции с кэшбэком. Мне вернули только половину обещанной суммы! Оператор: Сожалею, что возникло такое недопонимание. Давайте уточним условия акции, при которых вы рассчитывали на определенную сумму кэшбэка. Возможно, часть покупок не подпадала под действие акционного предложения согласно ограничени"}, 
]
}

ru_en_cat_names = {"Текущие (расчетные) счета": "current_acc",
"Депозиты (вклады)": "deposits",
"Кредиты": "credits",
"Кредитные карты": "credit_cards",
"Дебетовые карты": "debit_cards",
"Денежные переводы и платежи (внутренние и международные)": "payments",
"Инвестиционные продукты (брокерские услуги, ПИФы, доверительное управление)": "investments",
"Банковские гарантии и аккредитивы": "garant",
"Услуги инкассации и хранения ценностей": "colect_serv",
"Зарплатные проекты для организаций": "salary",
"Приложение банка": "bank_app",
"Автокредиты": "car_loans",
"Кэшбэк": "cashback",}
len(vocab_dialogs)

In [ ]:
# Генерируем эмбеддинги по категориям
cat_names = []
cat_embeddings = []
pos_embeddings = []
neg_embeddings = []
for type_dialogs in vocab_dialogs:
    print(type_dialogs)
    cat_names.append(ru_en_cat_names[type_dialogs])
    pos = list(vocab_dialogs[type_dialogs][0].values())[0]
    neg = list(vocab_dialogs[type_dialogs][1].values())[0]
    # Формируем эмбеддиги
    pos_emb = bertmodel.encode(pos)
    neg_emb = bertmodel.encode(neg)
    posneg_emb = bertmodel.encode(pos+neg)
    posneg_mean_emb = np.array([pos_emb, neg_emb]).mean(axis=0)
    
    cat_embeddings.append(pos_emb)
    cat_embeddings.append(neg_emb)
    cat_embeddings.append(posneg_emb)
    cat_embeddings.append(posneg_mean_emb)
    pos_embeddings.append(pos_emb)
    neg_embeddings.append(neg_emb)

pos_embeddings = np.array(pos_embeddings)
neg_embeddings = np.array(neg_embeddings)

cat_embeddings.append(pos_embeddings.mean(axis=0))
cat_embeddings.append(neg_embeddings.mean(axis=0))
cat_embeddings = np.array(cat_embeddings)

cat_embeddings.shape

In [ ]:
# Формируем словарь названий фичей
columns_cat_names = []
for cat_name in cat_names:
    columns_cat_names.append(f'pos_{cat_name}_emb')
    columns_cat_names.append(f'neg_{cat_name}_emb')
    columns_cat_names.append(f'posneg_{cat_name}_emb')
    columns_cat_names.append(f'posneg_mean_{cat_name}_emb')        
columns_cat_names += ['positive_emb', 'negative_emb']
len(columns_cat_names)

In [ ]:
%%time
# Производим расчет косинусного расстояния
result_cos_sim = cosine_similarity(embeds, cat_embeddings)
result_cos_sim.shape

In [ ]:
# Сохраняем косинунсое расстояние в фичах для каждого диалога
all_dialogs_df[columns_cat_names] = result_cos_sim

In [ ]:
all_dialogs_df = all_dialogs_df[['client_id', 'event_time', 'embedding']]
all_dialogs_df.shape

In [ ]:
assert Falsem, "Ксоинсусное сходство для хороших и плохих проудктов"

In [ ]:
%%time
# загружаем сэмплерированные данные, для которых будем рассчитывать фичи
smpl_Client_Month_df = pq.read_table(PATH_DATASET_OUTPUT + 'result_sample_Client_Month_df_12_06_2024.parquet').to_pandas()
smpl_Client_Month_df = smpl_Client_Month_df.set_index(['client_id', 'report_next_end'])
smpl_Client_Month_df.shape

In [ ]:
# Добавляем временные колонки в сэпмплирвоанные данные, чтобы легче искать по индексу и отдельно по клиенту и по месяцу
smpl_Client_Month_df = smpl_Client_Month_df.reset_index()
smpl_Client_Month_df['col_client_id'] = smpl_Client_Month_df['client_id']
smpl_Client_Month_df['col_report_next_end'] = smpl_Client_Month_df['report_next_end']
smpl_Client_Month_df = smpl_Client_Month_df.set_index(['client_id', 'report_next_end'])
smpl_Client_Month_df.shape

### Генерируем доп фичи по косинусному сходству

In [ ]:
# columns_cat_names

In [ ]:
%%time
def calc_embedding_stats(data):
    data = select_mon_dlg_df

    # Группируем данные по client_id
    stats_dialogs = data.groupby('client_id').agg(
            sum_pos_current_acc_emb = ('pos_current_acc_emb', sum),
            sum_neg_current_acc_emb = ('neg_current_acc_emb', sum),
            sum_posneg_current_acc_emb = ('posneg_current_acc_emb', sum),
            sum_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', sum),
            sum_pos_deposits_emb = ('pos_deposits_emb', sum),
            sum_neg_deposits_emb = ('neg_deposits_emb', sum),
            sum_posneg_deposits_emb = ('posneg_deposits_emb', sum),
            sum_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', sum),
            sum_pos_credits_emb = ('pos_credits_emb', sum),
            sum_neg_credits_emb = ('neg_credits_emb', sum),
            sum_posneg_credits_emb = ('posneg_credits_emb', sum),
            sum_posneg_mean_credits_emb = ('posneg_mean_credits_emb', sum),
            sum_pos_credit_cards_emb = ('pos_credit_cards_emb', sum),
            sum_neg_credit_cards_emb = ('neg_credit_cards_emb', sum),
            sum_posneg_credit_cards_emb = ('posneg_credit_cards_emb', sum),
            sum_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', sum),
            sum_pos_debit_cards_emb = ('pos_debit_cards_emb', sum),
            sum_neg_debit_cards_emb = ('neg_debit_cards_emb', sum),
            sum_posneg_debit_cards_emb = ('posneg_debit_cards_emb', sum),
            sum_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', sum),
            sum_pos_payments_emb = ('pos_payments_emb', sum),
            sum_neg_payments_emb = ('neg_payments_emb', sum),
            sum_posneg_payments_emb = ('posneg_payments_emb', sum),
            sum_posneg_mean_payments_emb = ('posneg_mean_payments_emb', sum),
            sum_pos_investments_emb = ('pos_investments_emb', sum),
            sum_neg_investments_emb = ('neg_investments_emb', sum),
            sum_posneg_investments_emb = ('posneg_investments_emb', sum),
            sum_posneg_mean_investments_emb = ('posneg_mean_investments_emb', sum),
            sum_pos_garant_emb = ('pos_garant_emb', sum),
            sum_neg_garant_emb = ('neg_garant_emb', sum),
            sum_posneg_garant_emb = ('posneg_garant_emb', sum),
            sum_posneg_mean_garant_emb = ('posneg_mean_garant_emb', sum),
            sum_pos_colect_serv_emb = ('pos_colect_serv_emb', sum),
            sum_neg_colect_serv_emb = ('neg_colect_serv_emb', sum),
            sum_posneg_colect_serv_emb = ('posneg_colect_serv_emb', sum),
            sum_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', sum),
            sum_pos_salary_emb = ('pos_salary_emb', sum),
            sum_neg_salary_emb = ('neg_salary_emb', sum),
            sum_posneg_salary_emb = ('posneg_salary_emb', sum),
            sum_posneg_mean_salary_emb = ('posneg_mean_salary_emb', sum),
            sum_pos_bank_app_emb = ('pos_bank_app_emb', sum),
            sum_neg_bank_app_emb = ('neg_bank_app_emb', sum),
            sum_posneg_bank_app_emb = ('posneg_bank_app_emb', sum),
            sum_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', sum),
            sum_pos_car_loans_emb = ('pos_car_loans_emb', sum),
            sum_neg_car_loans_emb = ('neg_car_loans_emb', sum),
            sum_posneg_car_loans_emb = ('posneg_car_loans_emb', sum),
            sum_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', sum),
            sum_pos_cashback_emb = ('pos_cashback_emb', sum),
            sum_neg_cashback_emb = ('neg_cashback_emb', sum),
            sum_posneg_cashback_emb = ('posneg_cashback_emb', sum),
            sum_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', sum),
            sum_positive_emb = ('positive_emb', sum),
            sum_negative_emb = ('negative_emb', sum),
    
    
            mean_pos_current_acc_emb = ('pos_current_acc_emb', np.mean),
            mean_neg_current_acc_emb = ('neg_current_acc_emb', np.mean),
            mean_posneg_current_acc_emb = ('posneg_current_acc_emb', np.mean),
            mean_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', np.mean),
            mean_pos_deposits_emb = ('pos_deposits_emb', np.mean),
            mean_neg_deposits_emb = ('neg_deposits_emb', np.mean),
            mean_posneg_deposits_emb = ('posneg_deposits_emb', np.mean),
            mean_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', np.mean),
            mean_pos_credits_emb = ('pos_credits_emb', np.mean),
            mean_neg_credits_emb = ('neg_credits_emb', np.mean),
            mean_posneg_credits_emb = ('posneg_credits_emb', np.mean),
            mean_posneg_mean_credits_emb = ('posneg_mean_credits_emb', np.mean),
            mean_pos_credit_cards_emb = ('pos_credit_cards_emb', np.mean),
            mean_neg_credit_cards_emb = ('neg_credit_cards_emb', np.mean),
            mean_posneg_credit_cards_emb = ('posneg_credit_cards_emb', np.mean),
            mean_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', np.mean),
            mean_pos_debit_cards_emb = ('pos_debit_cards_emb', np.mean),
            mean_neg_debit_cards_emb = ('neg_debit_cards_emb', np.mean),
            mean_posneg_debit_cards_emb = ('posneg_debit_cards_emb', np.mean),
            mean_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', np.mean),
            mean_pos_payments_emb = ('pos_payments_emb', np.mean),
            mean_neg_payments_emb = ('neg_payments_emb', np.mean),
            mean_posneg_payments_emb = ('posneg_payments_emb', np.mean),
            mean_posneg_mean_payments_emb = ('posneg_mean_payments_emb', np.mean),
            mean_pos_investments_emb = ('pos_investments_emb', np.mean),
            mean_neg_investments_emb = ('neg_investments_emb', np.mean),
            mean_posneg_investments_emb = ('posneg_investments_emb', np.mean),
            mean_posneg_mean_investments_emb = ('posneg_mean_investments_emb', np.mean),
            mean_pos_garant_emb = ('pos_garant_emb', np.mean),
            mean_neg_garant_emb = ('neg_garant_emb', np.mean),
            mean_posneg_garant_emb = ('posneg_garant_emb', np.mean),
            mean_posneg_mean_garant_emb = ('posneg_mean_garant_emb', np.mean),
            mean_pos_colect_serv_emb = ('pos_colect_serv_emb', np.mean),
            mean_neg_colect_serv_emb = ('neg_colect_serv_emb', np.mean),
            mean_posneg_colect_serv_emb = ('posneg_colect_serv_emb', np.mean),
            mean_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', np.mean),
            mean_pos_salary_emb = ('pos_salary_emb', np.mean),
            mean_neg_salary_emb = ('neg_salary_emb', np.mean),
            mean_posneg_salary_emb = ('posneg_salary_emb', np.mean),
            mean_posneg_mean_salary_emb = ('posneg_mean_salary_emb', np.mean),
            mean_pos_bank_app_emb = ('pos_bank_app_emb', np.mean),
            mean_neg_bank_app_emb = ('neg_bank_app_emb', np.mean),
            mean_posneg_bank_app_emb = ('posneg_bank_app_emb', np.mean),
            mean_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', np.mean),
            mean_pos_car_loans_emb = ('pos_car_loans_emb', np.mean),
            mean_neg_car_loans_emb = ('neg_car_loans_emb', np.mean),
            mean_posneg_car_loans_emb = ('posneg_car_loans_emb', np.mean),
            mean_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', np.mean),
            mean_pos_cashback_emb = ('pos_cashback_emb', np.mean),
            mean_neg_cashback_emb = ('neg_cashback_emb', np.mean),
            mean_posneg_cashback_emb = ('posneg_cashback_emb', np.mean),
            mean_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', np.mean),
            mean_positive_emb = ('positive_emb', np.mean),
            mean_negative_emb = ('negative_emb', np.mean),

            min_pos_current_acc_emb = ('pos_current_acc_emb', min),
            min_neg_current_acc_emb = ('neg_current_acc_emb', min),
            min_posneg_current_acc_emb = ('posneg_current_acc_emb', min),
            min_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', min),
            min_pos_deposits_emb = ('pos_deposits_emb', min),
            min_neg_deposits_emb = ('neg_deposits_emb', min),
            min_posneg_deposits_emb = ('posneg_deposits_emb', min),
            min_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', min),
            min_pos_credits_emb = ('pos_credits_emb', min),
            min_neg_credits_emb = ('neg_credits_emb', min),
            min_posneg_credits_emb = ('posneg_credits_emb', min),
            min_posneg_mean_credits_emb = ('posneg_mean_credits_emb', min),
            min_pos_credit_cards_emb = ('pos_credit_cards_emb', min),
            min_neg_credit_cards_emb = ('neg_credit_cards_emb', min),
            min_posneg_credit_cards_emb = ('posneg_credit_cards_emb', min),
            min_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', min),
            min_pos_debit_cards_emb = ('pos_debit_cards_emb', min),
            min_neg_debit_cards_emb = ('neg_debit_cards_emb', min),
            min_posneg_debit_cards_emb = ('posneg_debit_cards_emb', min),
            min_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', min),
            min_pos_payments_emb = ('pos_payments_emb', min),
            min_neg_payments_emb = ('neg_payments_emb', min),
            min_posneg_payments_emb = ('posneg_payments_emb', min),
            min_posneg_mean_payments_emb = ('posneg_mean_payments_emb', min),
            min_pos_investments_emb = ('pos_investments_emb', min),
            min_neg_investments_emb = ('neg_investments_emb', min),
            min_posneg_investments_emb = ('posneg_investments_emb', min),
            min_posneg_mean_investments_emb = ('posneg_mean_investments_emb', min),
            min_pos_garant_emb = ('pos_garant_emb', min),
            min_neg_garant_emb = ('neg_garant_emb', min),
            min_posneg_garant_emb = ('posneg_garant_emb', min),
            min_posneg_mean_garant_emb = ('posneg_mean_garant_emb', min),
            min_pos_colect_serv_emb = ('pos_colect_serv_emb', min),
            min_neg_colect_serv_emb = ('neg_colect_serv_emb', min),
            min_posneg_colect_serv_emb = ('posneg_colect_serv_emb', min),
            min_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', min),
            min_pos_salary_emb = ('pos_salary_emb', min),
            min_neg_salary_emb = ('neg_salary_emb', min),
            min_posneg_salary_emb = ('posneg_salary_emb', min),
            min_posneg_mean_salary_emb = ('posneg_mean_salary_emb', min),
            min_pos_bank_app_emb = ('pos_bank_app_emb', min),
            min_neg_bank_app_emb = ('neg_bank_app_emb', min),
            min_posneg_bank_app_emb = ('posneg_bank_app_emb', min),
            min_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', min),
            min_pos_car_loans_emb = ('pos_car_loans_emb', min),
            min_neg_car_loans_emb = ('neg_car_loans_emb', min),
            min_posneg_car_loans_emb = ('posneg_car_loans_emb', min),
            min_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', min),
            min_pos_cashback_emb = ('pos_cashback_emb', min),
            min_neg_cashback_emb = ('neg_cashback_emb', min),
            min_posneg_cashback_emb = ('posneg_cashback_emb', min),
            min_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', min),
            min_positive_emb = ('positive_emb', min),
            min_negative_emb = ('negative_emb', min),
    
    
            max_pos_current_acc_emb = ('pos_current_acc_emb', max),
            max_neg_current_acc_emb = ('neg_current_acc_emb', max),
            max_posneg_current_acc_emb = ('posneg_current_acc_emb', max),
            max_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', max),
            max_pos_deposits_emb = ('pos_deposits_emb', max),
            max_neg_deposits_emb = ('neg_deposits_emb', max),
            max_posneg_deposits_emb = ('posneg_deposits_emb', max),
            max_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', max),
            max_pos_credits_emb = ('pos_credits_emb', max),
            max_neg_credits_emb = ('neg_credits_emb', max),
            max_posneg_credits_emb = ('posneg_credits_emb', max),
            max_posneg_mean_credits_emb = ('posneg_mean_credits_emb', max),
            max_pos_credit_cards_emb = ('pos_credit_cards_emb', max),
            max_neg_credit_cards_emb = ('neg_credit_cards_emb', max),
            max_posneg_credit_cards_emb = ('posneg_credit_cards_emb', max),
            max_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', max),
            max_pos_debit_cards_emb = ('pos_debit_cards_emb', max),
            max_neg_debit_cards_emb = ('neg_debit_cards_emb', max),
            max_posneg_debit_cards_emb = ('posneg_debit_cards_emb', max),
            max_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', max),
            max_pos_payments_emb = ('pos_payments_emb', max),
            max_neg_payments_emb = ('neg_payments_emb', max),
            max_posneg_payments_emb = ('posneg_payments_emb', max),
            max_posneg_mean_payments_emb = ('posneg_mean_payments_emb', max),
            max_pos_investments_emb = ('pos_investments_emb', max),
            max_neg_investments_emb = ('neg_investments_emb', max),
            max_posneg_investments_emb = ('posneg_investments_emb', max),
            max_posneg_mean_investments_emb = ('posneg_mean_investments_emb', max),
            max_pos_garant_emb = ('pos_garant_emb', max),
            max_neg_garant_emb = ('neg_garant_emb', max),
            max_posneg_garant_emb = ('posneg_garant_emb', max),
            max_posneg_mean_garant_emb = ('posneg_mean_garant_emb', max),
            max_pos_colect_serv_emb = ('pos_colect_serv_emb', max),
            max_neg_colect_serv_emb = ('neg_colect_serv_emb', max),
            max_posneg_colect_serv_emb = ('posneg_colect_serv_emb', max),
            max_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', max),
            max_pos_salary_emb = ('pos_salary_emb', max),
            max_neg_salary_emb = ('neg_salary_emb', max),
            max_posneg_salary_emb = ('posneg_salary_emb', max),
            max_posneg_mean_salary_emb = ('posneg_mean_salary_emb', max),
            max_pos_bank_app_emb = ('pos_bank_app_emb', max),
            max_neg_bank_app_emb = ('neg_bank_app_emb', max),
            max_posneg_bank_app_emb = ('posneg_bank_app_emb', max),
            max_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', max),
            max_pos_car_loans_emb = ('pos_car_loans_emb', max),
            max_neg_car_loans_emb = ('neg_car_loans_emb', max),
            max_posneg_car_loans_emb = ('posneg_car_loans_emb', max),
            max_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', max),
            max_pos_cashback_emb = ('pos_cashback_emb', max),
            max_neg_cashback_emb = ('neg_cashback_emb', max),
            max_posneg_cashback_emb = ('posneg_cashback_emb', max),
            max_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', max),
            max_positive_emb = ('positive_emb', max),
            max_negative_emb = ('negative_emb', max),    
    
            std_pos_current_acc_emb = ('pos_current_acc_emb', np.std),
            std_neg_current_acc_emb = ('neg_current_acc_emb', np.std),
            std_posneg_current_acc_emb = ('posneg_current_acc_emb', np.std),
            std_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', np.std),
            std_pos_deposits_emb = ('pos_deposits_emb', np.std),
            std_neg_deposits_emb = ('neg_deposits_emb', np.std),
            std_posneg_deposits_emb = ('posneg_deposits_emb', np.std),
            std_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', np.std),
            std_pos_credits_emb = ('pos_credits_emb', np.std),
            std_neg_credits_emb = ('neg_credits_emb', np.std),
            std_posneg_credits_emb = ('posneg_credits_emb', np.std),
            std_posneg_mean_credits_emb = ('posneg_mean_credits_emb', np.std),
            std_pos_credit_cards_emb = ('pos_credit_cards_emb', np.std),
            std_neg_credit_cards_emb = ('neg_credit_cards_emb', np.std),
            std_posneg_credit_cards_emb = ('posneg_credit_cards_emb', np.std),
            std_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', np.std),
            std_pos_debit_cards_emb = ('pos_debit_cards_emb', np.std),
            std_neg_debit_cards_emb = ('neg_debit_cards_emb', np.std),
            std_posneg_debit_cards_emb = ('posneg_debit_cards_emb', np.std),
            std_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', np.std),
            std_pos_payments_emb = ('pos_payments_emb', np.std),
            std_neg_payments_emb = ('neg_payments_emb', np.std),
            std_posneg_payments_emb = ('posneg_payments_emb', np.std),
            std_posneg_mean_payments_emb = ('posneg_mean_payments_emb', np.std),
            std_pos_investments_emb = ('pos_investments_emb', np.std),
            std_neg_investments_emb = ('neg_investments_emb', np.std),
            std_posneg_investments_emb = ('posneg_investments_emb', np.std),
            std_posneg_mean_investments_emb = ('posneg_mean_investments_emb', np.std),
            std_pos_garant_emb = ('pos_garant_emb', np.std),
            std_neg_garant_emb = ('neg_garant_emb', np.std),
            std_posneg_garant_emb = ('posneg_garant_emb', np.std),
            std_posneg_mean_garant_emb = ('posneg_mean_garant_emb', np.std),
            std_pos_colect_serv_emb = ('pos_colect_serv_emb', np.std),
            std_neg_colect_serv_emb = ('neg_colect_serv_emb', np.std),
            std_posneg_colect_serv_emb = ('posneg_colect_serv_emb', np.std),
            std_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', np.std),
            std_pos_salary_emb = ('pos_salary_emb', np.std),
            std_neg_salary_emb = ('neg_salary_emb', np.std),
            std_posneg_salary_emb = ('posneg_salary_emb', np.std),
            std_posneg_mean_salary_emb = ('posneg_mean_salary_emb', np.std),
            std_pos_bank_app_emb = ('pos_bank_app_emb', np.std),
            std_neg_bank_app_emb = ('neg_bank_app_emb', np.std),
            std_posneg_bank_app_emb = ('posneg_bank_app_emb', np.std),
            std_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', np.std),
            std_pos_car_loans_emb = ('pos_car_loans_emb', np.std),
            std_neg_car_loans_emb = ('neg_car_loans_emb', np.std),
            std_posneg_car_loans_emb = ('posneg_car_loans_emb', np.std),
            std_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', np.std),
            std_pos_cashback_emb = ('pos_cashback_emb', np.std),
            std_neg_cashback_emb = ('neg_cashback_emb', np.std),
            std_posneg_cashback_emb = ('posneg_cashback_emb', np.std),
            std_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', np.std),
            std_positive_emb = ('positive_emb', np.std),
            std_negative_emb = ('negative_emb', np.std),    
            )
    return stats_dialogs.reset_index()

In [ ]:
%%time
# Формируем фичи по таргету
start_date = datetime(2022, 1, 1, 0, 0, 0)
# end_date = datetime(2023, 1, 1, 0, 0, 0)
end_date = all_dialogs_df['event_time'].max()

# Итоговый датасет 
union_cos_dialogs_by_clients_df = pd.DataFrame()

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')
    # Начальная дата за прошедшие полгода
    begin_date = end_date - relativedelta(months=6) - relativedelta(days=1)
    # Определяем только тех клиентов которые есть в сэмлере для указанной отчетной даты
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    good_slct_clients = smpl_Client_Month_df[smpl_Client_Month_df['col_report_next_end'] == report_next_end]['col_client_id'].unique()

    # Берем данные за последний месяц и фильтруем по нужным клиентам 
    select_mon_dlg_df = all_dialogs_df[all_dialogs_df['event_time'].between(start_date, end_date)]
    select_mon_dlg_df = select_mon_dlg_df[select_mon_dlg_df['client_id'].isin(good_slct_clients)]

    # Берем данные за последние полгода начиная от даты begin_date и фильтруем по нужным клиентам 
    select_ftime_dlg_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
    select_ftime_dlg_df = select_ftime_dlg_df[select_ftime_dlg_df['client_id'].isin(good_slct_clients)]
    
    print(select_mon_dlg_df.shape, select_ftime_dlg_df.shape)
    
    
#     select_mon_current_df = all_dialogs_df[all_dialogs_df['event_time'].between(start_date, end_date)]
#     select_mon_full_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
#     print(select_mon_current_df.shape, select_mon_full_df.shape)
    
    # client_agg_df = uniq_clients_df.copy()
#     client_agg_df = select_mon_full_df[['client_id']].drop_duplicates().copy()
#     report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
#     client_agg_df['report_next_end'] = report_next_end
#     client_agg_df = client_agg_df.set_index('client_id')    

    pop_mon_stat_dialogs_df = calc_embedding_stats(select_mon_dlg_df).fillna(0)
    pop_mon_stat_dialogs_df['report_next_end'] = report_next_end
    pop_mon_stat_dialogs_df = pop_mon_stat_dialogs_df.set_index(['client_id', 'report_next_end']).add_prefix('dlg_')
    
    pop_ftime_stat_dialogs_df = calc_embedding_stats(select_ftime_dlg_df).fillna(0)
    pop_ftime_stat_dialogs_df['report_next_end'] = report_next_end
    pop_ftime_stat_dialogs_df = pop_ftime_stat_dialogs_df.set_index(['client_id', 'report_next_end']).add_prefix('ftime_dlg_')
    
    # Объединяем фичи по месяцам и за полгода
    pop_ftime_stat_dialogs_df = pop_ftime_stat_dialogs_df.merge(pop_mon_stat_dialogs_df, left_index=True, right_index=True, how='left')
    
    # Сводим в единый датафрейм
    union_cos_dialogs_by_clients_df = pd.concat([union_cos_dialogs_by_clients_df, pop_ftime_stat_dialogs_df])
    
    start_date = start_date + relativedelta(months=1)
union_cos_dialogs_by_clients_df.shape

In [ ]:
%%time
# Сохраняем статистику по косинусной близости
union_cos_dialogs_by_clients_df.to_parquet(PATH_DATASET_OUTPUT + 'cos_dialogs_by_clients_df_13_06_2024.parquet')

### Разбивка эмбеддинга на подгруппы и рассчет статистик отдельно для них 

In [ ]:
length_embedding = 768
part_size = 64 # должно быть одно из делителей 768: 1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 256, 384, 768
count_parts_emb = math.ceil(768/part_size)
columns_part = []
for i in range(count_parts_emb):
    columns_part.append(f'{i+1}_part_emb')
len(columns_part)

In [ ]:
all_dialogs_df = all_dialogs_df[['client_id', 'event_time', 'embedding']]
all_dialogs_df.shape

In [ ]:
def calc_part_emb_stats(data):
    # Группируем данные по client_id
    stats_part_emds = data.groupby('client_id').agg(
        min__min_1_part_emb = ('min_1_part_emb', min),
        min__max_1_part_emb = ('max_1_part_emb', min),
        min__sum_1_part_emb = ('sum_1_part_emb', min),
        min__std_1_part_emb = ('std_1_part_emb', min),
        min__mean_1_part_emb = ('mean_1_part_emb', min),
        min__min_2_part_emb = ('min_2_part_emb', min),
        min__max_2_part_emb = ('max_2_part_emb', min),
        min__sum_2_part_emb = ('sum_2_part_emb', min),
        min__std_2_part_emb = ('std_2_part_emb', min),
        min__mean_2_part_emb = ('mean_2_part_emb', min),
        min__min_3_part_emb = ('min_3_part_emb', min),
        min__max_3_part_emb = ('max_3_part_emb', min),
        min__sum_3_part_emb = ('sum_3_part_emb', min),
        min__std_3_part_emb = ('std_3_part_emb', min),
        min__mean_3_part_emb = ('mean_3_part_emb', min),
        min__min_4_part_emb = ('min_4_part_emb', min),
        min__max_4_part_emb = ('max_4_part_emb', min),
        min__sum_4_part_emb = ('sum_4_part_emb', min),
        min__std_4_part_emb = ('std_4_part_emb', min),
        min__mean_4_part_emb = ('mean_4_part_emb', min),
        min__min_5_part_emb = ('min_5_part_emb', min),
        min__max_5_part_emb = ('max_5_part_emb', min),
        min__sum_5_part_emb = ('sum_5_part_emb', min),
        min__std_5_part_emb = ('std_5_part_emb', min),
        min__mean_5_part_emb = ('mean_5_part_emb', min),
        min__min_6_part_emb = ('min_6_part_emb', min),
        min__max_6_part_emb = ('max_6_part_emb', min),
        min__sum_6_part_emb = ('sum_6_part_emb', min),
        min__std_6_part_emb = ('std_6_part_emb', min),
        min__mean_6_part_emb = ('mean_6_part_emb', min),
        min__min_7_part_emb = ('min_7_part_emb', min),
        min__max_7_part_emb = ('max_7_part_emb', min),
        min__sum_7_part_emb = ('sum_7_part_emb', min),
        min__std_7_part_emb = ('std_7_part_emb', min),
        min__mean_7_part_emb = ('mean_7_part_emb', min),
        min__min_8_part_emb = ('min_8_part_emb', min),
        min__max_8_part_emb = ('max_8_part_emb', min),
        min__sum_8_part_emb = ('sum_8_part_emb', min),
        min__std_8_part_emb = ('std_8_part_emb', min),
        min__mean_8_part_emb = ('mean_8_part_emb', min),
        min__min_9_part_emb = ('min_9_part_emb', min),
        min__max_9_part_emb = ('max_9_part_emb', min),
        min__sum_9_part_emb = ('sum_9_part_emb', min),
        min__std_9_part_emb = ('std_9_part_emb', min),
        min__mean_9_part_emb = ('mean_9_part_emb', min),
        min__min_10_part_emb = ('min_10_part_emb', min),
        min__max_10_part_emb = ('max_10_part_emb', min),
        min__sum_10_part_emb = ('sum_10_part_emb', min),
        min__std_10_part_emb = ('std_10_part_emb', min),
        min__mean_10_part_emb = ('mean_10_part_emb', min),
        min__min_11_part_emb = ('min_11_part_emb', min),
        min__max_11_part_emb = ('max_11_part_emb', min),
        min__sum_11_part_emb = ('sum_11_part_emb', min),
        min__std_11_part_emb = ('std_11_part_emb', min),
        min__mean_11_part_emb = ('mean_11_part_emb', min),
        min__min_12_part_emb = ('min_12_part_emb', min),
        min__max_12_part_emb = ('max_12_part_emb', min),
        min__sum_12_part_emb = ('sum_12_part_emb', min),
        min__std_12_part_emb = ('std_12_part_emb', min),
        min__mean_12_part_emb = ('mean_12_part_emb', min),

        max__min_1_part_emb = ('min_1_part_emb', max),
        max__max_1_part_emb = ('max_1_part_emb', max),
        max__sum_1_part_emb = ('sum_1_part_emb', max),
        max__std_1_part_emb = ('std_1_part_emb', max),
        max__mean_1_part_emb = ('mean_1_part_emb', max),
        max__min_2_part_emb = ('min_2_part_emb', max),
        max__max_2_part_emb = ('max_2_part_emb', max),
        max__sum_2_part_emb = ('sum_2_part_emb', max),
        max__std_2_part_emb = ('std_2_part_emb', max),
        max__mean_2_part_emb = ('mean_2_part_emb', max),
        max__min_3_part_emb = ('min_3_part_emb', max),
        max__max_3_part_emb = ('max_3_part_emb', max),
        max__sum_3_part_emb = ('sum_3_part_emb', max),
        max__std_3_part_emb = ('std_3_part_emb', max),
        max__mean_3_part_emb = ('mean_3_part_emb', max),
        max__min_4_part_emb = ('min_4_part_emb', max),
        max__max_4_part_emb = ('max_4_part_emb', max),
        max__sum_4_part_emb = ('sum_4_part_emb', max),
        max__std_4_part_emb = ('std_4_part_emb', max),
        max__mean_4_part_emb = ('mean_4_part_emb', max),
        max__min_5_part_emb = ('min_5_part_emb', max),
        max__max_5_part_emb = ('max_5_part_emb', max),
        max__sum_5_part_emb = ('sum_5_part_emb', max),
        max__std_5_part_emb = ('std_5_part_emb', max),
        max__mean_5_part_emb = ('mean_5_part_emb', max),
        max__min_6_part_emb = ('min_6_part_emb', max),
        max__max_6_part_emb = ('max_6_part_emb', max),
        max__sum_6_part_emb = ('sum_6_part_emb', max),
        max__std_6_part_emb = ('std_6_part_emb', max),
        max__mean_6_part_emb = ('mean_6_part_emb', max),
        max__min_7_part_emb = ('min_7_part_emb', max),
        max__max_7_part_emb = ('max_7_part_emb', max),
        max__sum_7_part_emb = ('sum_7_part_emb', max),
        max__std_7_part_emb = ('std_7_part_emb', max),
        max__mean_7_part_emb = ('mean_7_part_emb', max),
        max__min_8_part_emb = ('min_8_part_emb', max),
        max__max_8_part_emb = ('max_8_part_emb', max),
        max__sum_8_part_emb = ('sum_8_part_emb', max),
        max__std_8_part_emb = ('std_8_part_emb', max),
        max__mean_8_part_emb = ('mean_8_part_emb', max),
        max__min_9_part_emb = ('min_9_part_emb', max),
        max__max_9_part_emb = ('max_9_part_emb', max),
        max__sum_9_part_emb = ('sum_9_part_emb', max),
        max__std_9_part_emb = ('std_9_part_emb', max),
        max__mean_9_part_emb = ('mean_9_part_emb', max),
        max__min_10_part_emb = ('min_10_part_emb', max),
        max__max_10_part_emb = ('max_10_part_emb', max),
        max__sum_10_part_emb = ('sum_10_part_emb', max),
        max__std_10_part_emb = ('std_10_part_emb', max),
        max__mean_10_part_emb = ('mean_10_part_emb', max),
        max__min_11_part_emb = ('min_11_part_emb', max),
        max__max_11_part_emb = ('max_11_part_emb', max),
        max__sum_11_part_emb = ('sum_11_part_emb', max),
        max__std_11_part_emb = ('std_11_part_emb', max),
        max__mean_11_part_emb = ('mean_11_part_emb', max),
        max__min_12_part_emb = ('min_12_part_emb', max),
        max__max_12_part_emb = ('max_12_part_emb', max),
        max__sum_12_part_emb = ('sum_12_part_emb', max),
        max__std_12_part_emb = ('std_12_part_emb', max),
        max__mean_12_part_emb = ('mean_12_part_emb', max),

        mean__min_1_part_emb = ('min_1_part_emb', np.mean),
        mean__max_1_part_emb = ('max_1_part_emb', np.mean),
        mean__sum_1_part_emb = ('sum_1_part_emb', np.mean),
        mean__std_1_part_emb = ('std_1_part_emb', np.mean),
        mean__mean_1_part_emb = ('mean_1_part_emb', np.mean),
        mean__min_2_part_emb = ('min_2_part_emb', np.mean),
        mean__max_2_part_emb = ('max_2_part_emb', np.mean),
        mean__sum_2_part_emb = ('sum_2_part_emb', np.mean),
        mean__std_2_part_emb = ('std_2_part_emb', np.mean),
        mean__mean_2_part_emb = ('mean_2_part_emb', np.mean),
        mean__min_3_part_emb = ('min_3_part_emb', np.mean),
        mean__max_3_part_emb = ('max_3_part_emb', np.mean),
        mean__sum_3_part_emb = ('sum_3_part_emb', np.mean),
        mean__std_3_part_emb = ('std_3_part_emb', np.mean),
        mean__mean_3_part_emb = ('mean_3_part_emb', np.mean),
        mean__min_4_part_emb = ('min_4_part_emb', np.mean),
        mean__max_4_part_emb = ('max_4_part_emb', np.mean),
        mean__sum_4_part_emb = ('sum_4_part_emb', np.mean),
        mean__std_4_part_emb = ('std_4_part_emb', np.mean),
        mean__mean_4_part_emb = ('mean_4_part_emb', np.mean),
        mean__min_5_part_emb = ('min_5_part_emb', np.mean),
        mean__max_5_part_emb = ('max_5_part_emb', np.mean),
        mean__sum_5_part_emb = ('sum_5_part_emb', np.mean),
        mean__std_5_part_emb = ('std_5_part_emb', np.mean),
        mean__mean_5_part_emb = ('mean_5_part_emb', np.mean),
        mean__min_6_part_emb = ('min_6_part_emb', np.mean),
        mean__max_6_part_emb = ('max_6_part_emb', np.mean),
        mean__sum_6_part_emb = ('sum_6_part_emb', np.mean),
        mean__std_6_part_emb = ('std_6_part_emb', np.mean),
        mean__mean_6_part_emb = ('mean_6_part_emb', np.mean),
        mean__min_7_part_emb = ('min_7_part_emb', np.mean),
        mean__max_7_part_emb = ('max_7_part_emb', np.mean),
        mean__sum_7_part_emb = ('sum_7_part_emb', np.mean),
        mean__std_7_part_emb = ('std_7_part_emb', np.mean),
        mean__mean_7_part_emb = ('mean_7_part_emb', np.mean),
        mean__min_8_part_emb = ('min_8_part_emb', np.mean),
        mean__max_8_part_emb = ('max_8_part_emb', np.mean),
        mean__sum_8_part_emb = ('sum_8_part_emb', np.mean),
        mean__std_8_part_emb = ('std_8_part_emb', np.mean),
        mean__mean_8_part_emb = ('mean_8_part_emb', np.mean),
        mean__min_9_part_emb = ('min_9_part_emb', np.mean),
        mean__max_9_part_emb = ('max_9_part_emb', np.mean),
        mean__sum_9_part_emb = ('sum_9_part_emb', np.mean),
        mean__std_9_part_emb = ('std_9_part_emb', np.mean),
        mean__mean_9_part_emb = ('mean_9_part_emb', np.mean),
        mean__min_10_part_emb = ('min_10_part_emb', np.mean),
        mean__max_10_part_emb = ('max_10_part_emb', np.mean),
        mean__sum_10_part_emb = ('sum_10_part_emb', np.mean),
        mean__std_10_part_emb = ('std_10_part_emb', np.mean),
        mean__mean_10_part_emb = ('mean_10_part_emb', np.mean),
        mean__min_11_part_emb = ('min_11_part_emb', np.mean),
        mean__max_11_part_emb = ('max_11_part_emb', np.mean),
        mean__sum_11_part_emb = ('sum_11_part_emb', np.mean),
        mean__std_11_part_emb = ('std_11_part_emb', np.mean),
        mean__mean_11_part_emb = ('mean_11_part_emb', np.mean),
        mean__min_12_part_emb = ('min_12_part_emb', np.mean),
        mean__max_12_part_emb = ('max_12_part_emb', np.mean),
        mean__sum_12_part_emb = ('sum_12_part_emb', np.mean),
        mean__std_12_part_emb = ('std_12_part_emb', np.mean),
        mean__mean_12_part_emb = ('mean_12_part_emb', np.mean),

        sum__min_1_part_emb = ('min_1_part_emb', sum),
        sum__max_1_part_emb = ('max_1_part_emb', sum),
        sum__sum_1_part_emb = ('sum_1_part_emb', sum),
        sum__std_1_part_emb = ('std_1_part_emb', sum),
        sum__mean_1_part_emb = ('mean_1_part_emb', sum),
        sum__min_2_part_emb = ('min_2_part_emb', sum),
        sum__max_2_part_emb = ('max_2_part_emb', sum),
        sum__sum_2_part_emb = ('sum_2_part_emb', sum),
        sum__std_2_part_emb = ('std_2_part_emb', sum),
        sum__mean_2_part_emb = ('mean_2_part_emb', sum),
        sum__min_3_part_emb = ('min_3_part_emb', sum),
        sum__max_3_part_emb = ('max_3_part_emb', sum),
        sum__sum_3_part_emb = ('sum_3_part_emb', sum),
        sum__std_3_part_emb = ('std_3_part_emb', sum),
        sum__mean_3_part_emb = ('mean_3_part_emb', sum),
        sum__min_4_part_emb = ('min_4_part_emb', sum),
        sum__max_4_part_emb = ('max_4_part_emb', sum),
        sum__sum_4_part_emb = ('sum_4_part_emb', sum),
        sum__std_4_part_emb = ('std_4_part_emb', sum),
        sum__mean_4_part_emb = ('mean_4_part_emb', sum),
        sum__min_5_part_emb = ('min_5_part_emb', sum),
        sum__max_5_part_emb = ('max_5_part_emb', sum),
        sum__sum_5_part_emb = ('sum_5_part_emb', sum),
        sum__std_5_part_emb = ('std_5_part_emb', sum),
        sum__mean_5_part_emb = ('mean_5_part_emb', sum),
        sum__min_6_part_emb = ('min_6_part_emb', sum),
        sum__max_6_part_emb = ('max_6_part_emb', sum),
        sum__sum_6_part_emb = ('sum_6_part_emb', sum),
        sum__std_6_part_emb = ('std_6_part_emb', sum),
        sum__mean_6_part_emb = ('mean_6_part_emb', sum),
        sum__min_7_part_emb = ('min_7_part_emb', sum),
        sum__max_7_part_emb = ('max_7_part_emb', sum),
        sum__sum_7_part_emb = ('sum_7_part_emb', sum),
        sum__std_7_part_emb = ('std_7_part_emb', sum),
        sum__mean_7_part_emb = ('mean_7_part_emb', sum),
        sum__min_8_part_emb = ('min_8_part_emb', sum),
        sum__max_8_part_emb = ('max_8_part_emb', sum),
        sum__sum_8_part_emb = ('sum_8_part_emb', sum),
        sum__std_8_part_emb = ('std_8_part_emb', sum),
        sum__mean_8_part_emb = ('mean_8_part_emb', sum),
        sum__min_9_part_emb = ('min_9_part_emb', sum),
        sum__max_9_part_emb = ('max_9_part_emb', sum),
        sum__sum_9_part_emb = ('sum_9_part_emb', sum),
        sum__std_9_part_emb = ('std_9_part_emb', sum),
        sum__mean_9_part_emb = ('mean_9_part_emb', sum),
        sum__min_10_part_emb = ('min_10_part_emb', sum),
        sum__max_10_part_emb = ('max_10_part_emb', sum),
        sum__sum_10_part_emb = ('sum_10_part_emb', sum),
        sum__std_10_part_emb = ('std_10_part_emb', sum),
        sum__mean_10_part_emb = ('mean_10_part_emb', sum),
        sum__min_11_part_emb = ('min_11_part_emb', sum),
        sum__max_11_part_emb = ('max_11_part_emb', sum),
        sum__sum_11_part_emb = ('sum_11_part_emb', sum),
        sum__std_11_part_emb = ('std_11_part_emb', sum),
        sum__mean_11_part_emb = ('mean_11_part_emb', sum),
        sum__min_12_part_emb = ('min_12_part_emb', sum),
        sum__max_12_part_emb = ('max_12_part_emb', sum),
        sum__sum_12_part_emb = ('sum_12_part_emb', sum),
        sum__std_12_part_emb = ('std_12_part_emb', sum),
        sum__mean_12_part_emb = ('mean_12_part_emb', sum),        
    )

    return stats_part_emds.reset_index()


In [ ]:
%%time
# Формируем фичи по таргету
start_date = datetime(2022, 1, 1, 0, 0, 0)
end_date = all_dialogs_df['event_time'].max()

# Итоговый датасет 
union_split_emb_dialogs_by_clients_df = pd.DataFrame()

def split_emb(x):
    split_embeddings = [x[i*part_size:(i+1)*part_size] for i in range(count_parts_emb)]
    return split_embeddings

def calc_stat_by_one_emb(data):
    for col in columns_part:
        data[f'min_{col}'] = data[col].apply(lambda x: np.min(x, axis=0))
        data[f'max_{col}'] = data[col].apply(lambda x: np.max(x, axis=0))
        data[f'sum_{col}'] = data[col].apply(lambda x: np.sum(x, axis=0))
        data[f'std_{col}'] = data[col].apply(lambda x: np.std(x, axis=0))
        data[f'mean_{col}'] = data[col].apply(lambda x: np.mean(x, axis=0))    
    return data

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')
    # Начальная дата за прошедшие полгода
    begin_date = end_date - relativedelta(months=6) - relativedelta(days=1)
    # Определяем только тех клиентов которые есть в сэмлере для указанной отчетной даты
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    good_slct_clients = smpl_Client_Month_df[smpl_Client_Month_df['col_report_next_end'] == report_next_end]['col_client_id'].unique()

    # Берем данные за последний месяц и фильтруем по нужным клиентам 
    select_mon_dlg_df = all_dialogs_df[all_dialogs_df['event_time'].between(start_date, end_date)]
    select_mon_dlg_df = select_mon_dlg_df[select_mon_dlg_df['client_id'].isin(good_slct_clients)]

    # Берем данные за последние полгода начиная от даты begin_date и фильтруем по нужным клиентам 
    select_ftime_dlg_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
    select_ftime_dlg_df = select_ftime_dlg_df[select_ftime_dlg_df['client_id'].isin(good_slct_clients)]

    if len(select_ftime_dlg_df) == 0:
        start_date = start_date + relativedelta(months=1)
        continue

    print(select_mon_dlg_df.shape, select_ftime_dlg_df.shape)
    # stat_part_emb_dialogs_df = calc_part_emb_stats(select_mon_dlg_df).fillna(0)

    select_mon_dlg_df[columns_part] = select_mon_dlg_df['embedding'].apply(lambda x: pd.Series(split_emb(x)))    
    select_mon_dlg_df = calc_stat_by_one_emb(select_mon_dlg_df)
    
    mon_part_emb_stats_df = calc_part_emb_stats(select_mon_dlg_df).fillna(0)
    mon_part_emb_stats_df['report_next_end'] = report_next_end
    mon_part_emb_stats_df = mon_part_emb_stats_df.set_index(['client_id', 'report_next_end']).add_prefix('dlg_')

    
    select_ftime_dlg_df[columns_part] = select_ftime_dlg_df['embedding'].apply(lambda x: pd.Series(split_emb(x)))    
    select_ftime_dlg_df = calc_stat_by_one_emb(select_ftime_dlg_df)
    
    ftime_part_emb_stats_df = calc_part_emb_stats(select_ftime_dlg_df).fillna(0)
    ftime_part_emb_stats_df['report_next_end'] = report_next_end
    ftime_part_emb_stats_df = ftime_part_emb_stats_df.set_index(['client_id', 'report_next_end']).add_prefix('ftime_dlg_')

    # Объединяем фичи по месяцам и за полгода
    ftime_part_emb_stats_df = ftime_part_emb_stats_df.merge(mon_part_emb_stats_df, left_index=True, right_index=True, how='left')
    
    # Сводим в единый датафрейм
    union_split_emb_dialogs_by_clients_df = pd.concat([union_split_emb_dialogs_by_clients_df, ftime_part_emb_stats_df])
    
    start_date = start_date + relativedelta(months=1)
union_split_emb_dialogs_by_clients_df.shape

In [ ]:
np.array(all_dialogs_df['embedding'].values).shape

In [ ]:
np.vstack(tmp_dialogs_df.apply(np.array).values)

In [ ]:
# ftime_part_emb_stats_df.shape
# union_split_emb_dialogs_by_clients_df.shape

In [ ]:
%%time
# Сохраняем статистику по косинусной близости
union_split_emb_dialogs_by_clients_df.to_parquet(PATH_DATASET_OUTPUT + 'part_embed_dialogs_by_clients_df_13_06_2024.parquet')

In [ ]:
# list(select_mon_dlg_df.columns)
# mon_part_emb_stats_df
union_split_emb_dialogs_by_clients_df.shape

In [ ]:
tmp_df = select_ftime_dlg_df.copy()
def split_emb(x):
    split_embeddings = [x[i*part_size:(i+1)*part_size] for i in range(count_parts_emb)]
    return split_embeddings

tmp_df[columns_part] = tmp_df['embedding'].apply(lambda x: pd.Series(split_emb(x)))

In [ ]:
tmp_df

In [ ]:
columns_part

In [ ]:
# data['min_1_part_emb'] = data['1_part_emb'].apply(lambda x: np.min(x, axis=0))
# data

In [ ]:
data = tmp_df
for col in columns_part:
    print(col)
    data[f'min_{col}'] = data[col].apply(lambda x: np.min(x, axis=0))
    data[f'max_{col}'] = data[col].apply(lambda x: np.max(x, axis=0))
    data[f'sum_{col}'] = data[col].apply(lambda x: np.sum(x, axis=0))
    data[f'std_{col}'] = data[col].apply(lambda x: np.std(x, axis=0))
    data[f'mean_{col}'] = data[col].apply(lambda x: np.mean(x, axis=0))

In [ ]:
data

In [ ]:
np.array([[1,2],[2,3]])

In [ ]:
tmp_df

In [ ]:
# all_dialogs_df

In [ ]:
def calculate_embedding_stats(data):
    stats = pd.DataFrame()

    # Группируем данные по client_id
    grouped = data.groupby('client_id')['embedding'].apply(lambda x: np.array(x.tolist()))
    # Средний эмбеддинг диалога
    stats['mean_embedding'] = grouped.apply(lambda x: np.mean(x, axis=0))
    
    # Сумма
    stats['sum_embedding'] = grouped.apply(lambda x: np.sum(x))
    
    # Дисперсия эмбеддингов диалога
    stats['embedding_variance'] = grouped.apply(lambda x: np.var(x, axis=0).mean())

    # Минимальное/максимальное расстояние между эмбеддингами
    distances = grouped.apply(lambda x: squareform(pdist(x, 'euclidean')))
    stats['min_embedding_distance'] = distances.apply(np.min)
    stats['max_embedding_distance'] = distances.apply(np.max)

    # Средняя разница между соседними эмбеддингами
    stats['mean_consecutive_distance'] = grouped.apply(lambda x: np.mean([np.linalg.norm(x[i] - x[i-1]) for i in range(1, len(x))]))

    # Энтропия эмбеддингов диалога
    # stats['embedding_entropy'] = grouped.apply(lambda x: np.mean(-np.sum(np.log(np.var(x, axis=0)) * np.var(x, axis=0), axis=1)))
    stats['embedding_entropy'] = grouped.apply(lambda x: -np.sum(np.log(np.var(x, axis=0)) * np.var(x, axis=0)))

    # Длина диалога в эмбеддингах
    stats['embedding_path_length'] = grouped.apply(lambda x: np.sum([np.linalg.norm(x[i] - x[i-1]) for i in range(1, len(x))]))

    # Разложение на главные компоненты (PCA) эмбеддингов диалога
    stats['pca_embedding'] = grouped.apply(lambda x: PCA().fit_transform(x)[0])

    return stats

In [ ]:
%%time
# Формируем фичи по таргету
begin_date = datetime(2022, 1, 1, 0, 0, 0)
start_date = datetime(2022, 1, 1, 0, 0, 0)

end_date = datetime(2023, 1, 31, 0, 0, 0)

# Бланк-датафрейм с клиентами 
uniq_clients_df = all_dialogs_df[['client_id']].drop_duplicates()

# Итоговый датасет 
union_client_agg_df = pd.DataFrame()

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')    
    
    select_mon_current_df = all_dialogs_df[all_dialogs_df['event_time'].between(start_date, end_date)]
    select_mon_full_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
    print(select_mon_current_df.shape, select_mon_full_df.shape)
    
    # client_agg_df = uniq_clients_df.copy()
    client_agg_df = select_mon_full_df[['client_id']].drop_duplicates().copy()
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    client_agg_df['report_next_end'] = report_next_end
    client_agg_df = client_agg_df.set_index('client_id')    
    
    # Считаем статистики только за прошедший месяц
    stats_mon = calculate_embedding_stats(select_mon_current_df)
    # Считаем статистики за весь прошедший период
    stats_fulltime = calculate_embedding_stats(select_mon_full_df)
    
    client_agg_df = client_agg_df.merge(stats_mon, left_index=True, right_index=True, how='left')
    client_agg_df = client_agg_df.merge(stats_fulltime.add_prefix('fullt_'), left_index=True, right_index=True, how='left')
    
    union_client_agg_df = pd.concat([union_client_agg_df, client_agg_df])

    start_date = start_date + relativedelta(months=1)
union_client_agg_df.shape    

In [ ]:
%%time
# Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
def series_to_int(col_df:pd.Series):
    """
    Перевод в целочисленные типы
    """
    min_val = col_df.min()
    max_val = col_df.max()
    if min_val >= -128 and max_val <= 127:
        col_df = col_df.astype('int8')
    elif min_val >= -32768 and max_val <= 32767:
        col_df = col_df.astype('int16')
    elif min_val >= -2147483648 and max_val <= 2147483647:
        col_df = col_df.astype('int32')
    else:
        col_df = col_df.astype('int64')
    return col_df

def compression_df(df:pd.DataFrame(), datetime_cols:List[str]=[], category_cols:List[str]=[]):
    """
    Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
    """
    float64_cols = list(df.select_dtypes(include='float64'))  
    df[float64_cols] = df[float64_cols].astype('float32')
    for col in df.columns:
        if col in category_cols:
            df[col] = df[col].astype('category')
        elif col in datetime_cols:
            if df[col].dtypes == 'object':
                df[col] = pd.to_datetime(df[col])
        # Если колонка содержит числа 
        elif is_integer_dtype(df[col]):
            if df[col].dtypes == 'int8':
                continue
            else:
                df[col] = series_to_int(df[col])
        elif is_float_dtype(df[col]):
            # Возможно ли перевести в число
            if np.array_equal(df[col].fillna(0), df[col].fillna(0).astype(int)):
                df[col] = df[col].fillna(0)
                df[col] = series_to_int(df[col])
    return df
union_client_agg_df = compression_df(union_client_agg_df, 
                            datetime_cols=['report_end' ,'report_next_end'],
                           )
union_client_agg_df.shape

In [ ]:
%%time
# Сохраняем в файл оптимизированный файл 
union_client_agg_df.to_parquet(PATH_DATASET_OUTPUT + 'client_agg_dialog_09_06_2024.parquet')

In [ ]:
# %%time
# # Сохраняем в файл оптимизированный файл 
# # union_client_agg_df.to_parquet(PATH_DATASET_OUTPUT + 'client_agg_dialog_09_06_2024.parquet', engine='pyarrow')
# filename = 'sample_client_agg_dialog_09_06_2024'
# compression_options = dict(method='zip', archive_name=f'{filename}.csv')
# union_client_agg_df.to_csv(f'{filename}.zip', compression=compression_options)
# # union_client_agg_df.to_csv(PATH_DATASET_OUTPUT + 'client_agg_dialog_09_06_2024.csv')
# union_client_agg_df.shape

In [ ]:
# union_client_agg_df.info()

In [ ]:
# union_client_agg_df['report_next_end'].max()

In [ ]:
# ('2023-01-31 00:00:00')

In [ ]:
# stats#['mean_embedding'] = grouped.apply(lambda x: np.mean(x, axis=0))
# grouped.apply(lambda x: np.mean(x, axis=0))
# grouped.apply(lambda x: np.sum(x))

In [ ]:
# stats_fulltime['mean_embedding'] = stats_fulltime['mean_embedding'].fillna([[0.0]])
# stats_fulltime['pca_embedding'] = stats_fulltime['pca_embedding'].fillna([[0.0]])
# ser.apply(lambda x: [np.nan] if pd.isnull(x) else x)
# stats_fulltime = stats_fulltime.fillna(0)
# stats_mon.fillna(0)['pca_embedding']

In [ ]:
client_agg_df = uniq_clients_df.copy()
report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
client_agg_df['report_next_end'] = report_next_end
client_agg_df = client_agg_df.set_index('client_id')

In [ ]:
client_agg_df = client_agg_df.merge(stats_mon, left_index=True, right_index=True, how='left')
client_agg_df = client_agg_df.merge(stats_fulltime.add_prefix('fullt_'), left_index=True, right_index=True, how='left')
client_agg_df.shape

In [ ]:
client_agg_df

In [ ]:
stats_mon.shape
# stats_fulltime

In [ ]:
select_mon_current_df.groupby(by='client_id')['embedding'].agg({
    
})

In [ ]:
select_mon_current_df

In [ ]:
%%time



stats_df = calculate_embedding_stats(select_mon_current_df)

In [ ]:
stats_df

In [ ]:
data = select_mon_current_df.copy()
data.shape

In [ ]:
pd.DataFrame(grouped).reset_index()

In [ ]:
# Темпоральные признаки
stats['mean_time_between_events'] = grouped.apply(lambda x: np.mean([data.loc[data['client_id'] == client_id, 'event_time'].diff().dt.total_seconds()[i] for i in range(1, len(x))]))


In [ ]:
Все равно в строчке:
stats['mean_time_between_events'] = grouped.apply(lambda x, client_id: np.mean([data.loc[data['client_id'] == client_id, 'event_time'].diff().dt.total_seconds()[i] for i in range(1, len(x))]), client_id=data['client_id'])

Возникает ошибка:
setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [ ]:
%%time
# Темпоральные признаки
def get_mean_time_between_events(group):
    return np.mean(data[data['client_id'] == group.name]['event_time'].diff().dt.total_seconds()[1:])

stats['mean_time_between_events'] = data.groupby('client_id').progress_apply(get_mean_time_between_events)


In [ ]:
all_dialogs_df.info()

In [ ]:
%%time
# Загружаем диалоги клиентов
dialog_df = pq.read_table(PATH_DATASET_OUTPUT + 'compress_targets_08_06_2024.parquet').to_pandas()
# targets_df = targets_df.rename(columns={'mon': 'report_next_end'})
targets_df = targets_df.reset_index()
targets_df = targets_df[['client_id', 'mon', 'target_1', 'target_2', 'target_3', 'target_4']]
targets_df.shape

In [ ]:
%%time
# Загружаем факты продаж продуктов по трейн клиентам
targets_df = pq.read_table(PATH_DATASET_OUTPUT + 'compress_targets_08_06_2024.parquet').to_pandas()
# targets_df = targets_df.rename(columns={'mon': 'report_next_end'})
targets_df = targets_df.reset_index()
targets_df = targets_df[['client_id', 'mon', 'target_1', 'target_2', 'target_3', 'target_4']]
targets_df.shape

In [ ]:
# В данных встречаются дубли клиент+отчетный месяц. Там всегда нули, поэтому просто удаляем дубли 
targets_df = targets_df.drop_duplicates(subset=['client_id', 'mon'])
targets_df.shape

In [ ]:
%%time
# Рассчитываем факт приобретения клиентом когда-либо продукта 1 или 2/3/4
def get_group_targets(df:pd.DataFrame) -> pd.DataFrame:
    # Факт приобретения клиентом когда-либо продукта 1 или 2/3/4
    df['is_target'] = df[['target_1', 'target_2', 'target_3', 'target_4']].max(axis=1)
    
    # Расширеный факт приобретения клиентом когда-либо группы продуктов 
    df['is_target_1_2'] = df[['target_1', 'target_2']].max(axis=1)
    df['is_target_1_3'] = df[['target_1', 'target_3']].max(axis=1)
    df['is_target_1_4'] = df[['target_1', 'target_4']].max(axis=1)
    df['is_target_2_3'] = df[['target_2', 'target_3']].max(axis=1)
    df['is_target_2_4'] = df[['target_2', 'target_4']].max(axis=1)
    df['is_target_3_4'] = df[['target_3', 'target_4']].max(axis=1)

    df['is_target_123'] = df[['target_1', 'target_2', 'target_3']].max(axis=1)
    df['is_target_134'] = df[['target_1', 'target_3', 'target_4']].max(axis=1)
    df['is_target_124'] = df[['target_1', 'target_2', 'target_4']].max(axis=1)
    df['is_target_234'] = df[['target_2', 'target_3', 'target_4']].max(axis=1)
    
    # Второй расширеный факт приобретения клиентом когда-либо группы продуктов 
    df['is_target_1_and_2'] = np.where(df[['target_1', 'target_2']].sum(axis=1) == 2, 1,0)
    df['is_target_1_and_3'] = np.where(df[['target_1', 'target_3']].sum(axis=1) == 2, 1,0)
    df['is_target_1_and_4'] = np.where(df[['target_1', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_2_and_3'] = np.where(df[['target_2', 'target_3']].sum(axis=1) == 2, 1,0)
    df['is_target_2_and_4'] = np.where(df[['target_2', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_3_and_4'] = np.where(df[['target_3', 'target_4']].sum(axis=1) == 2, 1,0)
    
    df['is_target_and_123'] = np.where(df[['target_1', 'target_2', 'target_3']].sum(axis=1) == 2, 1,0)
    df['is_target_and_134'] = np.where(df[['target_1', 'target_3', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_and_124'] = np.where(df[['target_1', 'target_2', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_and_234'] = np.where(df[['target_2', 'target_3', 'target_4']].sum(axis=1) == 2, 1,0)    
    
    # кол-во купленных продуктов
    df['is_target_cnt'] = df[['target_1', 'target_2', 'target_3', 'target_4']].sum(axis=1)

    return df

targets_df = get_group_targets(targets_df)
targets_df.shape

In [ ]:
target_columns = ['target_1', 'target_2', 'target_3', 'target_4',
                  'is_target', 'is_target_1_2', 'is_target_1_3',
                  'is_target_1_4', 'is_target_2_3', 'is_target_2_4', 'is_target_3_4',
                  'is_target_1_and_2', 'is_target_1_and_3', 'is_target_1_and_4',
                  'is_target_2_and_3', 'is_target_2_and_4', 'is_target_3_and_4',
                  'is_target_123', 'is_target_134', 'is_target_124', 'is_target_234', 
                  'is_target_cnt']
len(target_columns)

In [ ]:
%%time
mon_targets_df = targets_df.groupby(by='mon').agg(
    sum_target_1 = ('target_1', sum), 
    sum_target_2 = ('target_2', sum), 
    sum_target_3 = ('target_3', sum), 
    sum_target_4 = ('target_4', sum), 
    sum_is_target = ('is_target', sum), 
    sum_is_target_1_2 = ('is_target_1_2', sum), 
    sum_is_target_1_3 = ('is_target_1_3', sum), 
    sum_is_target_1_4 = ('is_target_1_4', sum), 
    sum_is_target_2_3 = ('is_target_2_3', sum), 
    sum_is_target_2_4 = ('is_target_2_4', sum), 
    sum_is_target_3_4 = ('is_target_3_4', sum), 
    sum_is_target_1_and_2 = ('is_target_1_and_2', sum), 
    sum_is_target_1_and_3 = ('is_target_1_and_3', sum), 
    sum_is_target_1_and_4 = ('is_target_1_and_4', sum), 
    sum_is_target_2_and_3 = ('is_target_2_and_3', sum), 
    sum_is_target_2_and_4 = ('is_target_2_and_4', sum), 
    sum_is_target_3_and_4 = ('is_target_3_and_4', sum), 
    sum_is_target_123 = ('is_target_123', sum), 
    sum_is_target_134 = ('is_target_134', sum), 
    sum_is_target_124 = ('is_target_124', sum), 
    sum_is_target_234 = ('is_target_234', sum), 
    sum_is_target_cnt = ('is_target_cnt', sum),  
)
mon_targets_df = mon_targets_df.reset_index()
mon_targets_df['next_mon'] = mon_targets_df['mon'].shift(1)
mon_targets_df['pre_mon'] = mon_targets_df['mon'].shift(-1)

mon_targets_df.shape

In [ ]:
%%time
# Формируем фичи по таргету
begin_date = datetime(2022, 1, 1, 0, 0, 0)
start_date = datetime(2022, 1, 1, 0, 0, 0)

end_date = datetime(2023, 3, 31, 0, 0, 0)

# Бланк-датафрейм с клиентами 
uniq_clients_df = targets_df[['client_id']].drop_duplicates()
# Итоговый датасет 
union_client_agg_df = pd.DataFrame()

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')    
    select_mon_current_df = targets_df[targets_df['mon'].between(start_date, end_date)]
    select_mon_full_df = targets_df[targets_df['mon'].between(begin_date, end_date)]
    print(select_mon_current_df.shape, select_mon_full_df.shape)
    
    client_agg_df = uniq_clients_df.copy()
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    client_agg_df['report_next_end'] = report_next_end
    client_agg_df = client_agg_df.set_index('client_id')
    
    select_mon_full_df = select_mon_full_df.set_index('client_id')
    for cur_tar in ['target_1', 'target_2', 'target_3', 'target_4', 'is_target']:
        # Расчитываем даты первой и последней покупки продукта 
        min_max_date_buy = select_mon_full_df[select_mon_full_df[cur_tar] == 1].groupby(by='client_id').agg(
                                    first_day_buy = ('mon', min),
                                    last_day_buy = ('mon', max),
        )
        #break
        client_agg_df = client_agg_df.merge(min_max_date_buy, left_index=True, right_index=True, how='left')
        client_agg_df[f'days_first_buy_{cur_tar}'] = (client_agg_df['report_next_end'] - client_agg_df['first_day_buy']).dt.days
        client_agg_df[f'days_last_buy_{cur_tar}'] = (client_agg_df['report_next_end'] - client_agg_df['last_day_buy']).dt.days
        client_agg_df = client_agg_df.drop(columns=['first_day_buy', 'last_day_buy'])
        client_agg_df = client_agg_df.fillna(0)
    
    # Количество покупок продуктов за весь период
    client_agg_df = client_agg_df.merge(
            select_mon_full_df.groupby(by='client_id').agg(
                    sum_target_1_by_all_period = ('target_1', sum),
                    sum_target_2_by_all_period = ('target_2', sum),
                    sum_target_3_by_all_period = ('target_3', sum),
                    sum_target_4_by_all_period = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    
    # Доля покупок по продуктам
    client_agg_df['sum_all_target_by_all_period'] = client_agg_df[['sum_target_1_by_all_period', 'sum_target_2_by_all_period', 'sum_target_3_by_all_period', 'sum_target_4_by_all_period']].sum(axis=1)
    client_agg_df['prc_target_1by_all_trgs'] = (client_agg_df['sum_target_1_by_all_period'] / client_agg_df['sum_all_target_by_all_period']).fillna(0)
    client_agg_df['prc_target_2by_all_trgs'] = (client_agg_df['sum_target_2_by_all_period'] / client_agg_df['sum_all_target_by_all_period']).fillna(0)
    client_agg_df['prc_target_3by_all_trgs'] = (client_agg_df['sum_target_3_by_all_period'] / client_agg_df['sum_all_target_by_all_period']).fillna(0)
    client_agg_df['prc_target_4by_all_trgs'] = (client_agg_df['sum_target_4_by_all_period'] / client_agg_df['sum_all_target_by_all_period']).fillna(0)
    
    # Сколько в среднем в месяц клиент покупает продуктов 
    cnt_month = (end_date - begin_date).days / 30
    client_agg_df['mean_all_target_by_per_mon'] = client_agg_df['sum_all_target_by_all_period'] / cnt_month
    client_agg_df['mean_target_1_by_per_mon'] = client_agg_df['sum_target_1_by_all_period'] / cnt_month
    client_agg_df['mean_target_2_by_per_mon'] = client_agg_df['sum_target_2_by_all_period'] / cnt_month
    client_agg_df['mean_target_3_by_per_mon'] = client_agg_df['sum_target_3_by_all_period'] / cnt_month
    client_agg_df['mean_target_4_by_per_mon'] = client_agg_df['sum_target_4_by_all_period'] / cnt_month    
    
    # Количество покупок продуктов за 30 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=1)].groupby(by='client_id').agg(
                    sum_target_1_by_1_mon = ('target_1', sum),
                    sum_target_2_by_1_mon = ('target_2', sum),
                    sum_target_3_by_1_mon = ('target_3', sum),
                    sum_target_4_by_1_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    # Количество покупок продуктов за 60 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=2)].groupby(by='client_id').agg(
                    sum_target_1_by_2_mon = ('target_1', sum),
                    sum_target_2_by_2_mon = ('target_2', sum),
                    sum_target_3_by_2_mon = ('target_3', sum),
                    sum_target_4_by_2_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    
    # Количество покупок продуктов за 90 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=3)].groupby(by='client_id').agg(
                    sum_target_1_by_3_mon = ('target_1', sum),
                    sum_target_2_by_3_mon = ('target_2', sum),
                    sum_target_3_by_3_mon = ('target_3', sum),
                    sum_target_4_by_3_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    # Количество покупок продуктов за 120 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=4)].groupby(by='client_id').agg(
                    sum_target_1_by_4_mon = ('target_1', sum),
                    sum_target_2_by_4_mon = ('target_2', sum),
                    sum_target_3_by_4_mon = ('target_3', sum),
                    sum_target_4_by_4_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    # Количество покупок продуктов за 150 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=5)].groupby(by='client_id').agg(
                    sum_target_1_by_5_mon = ('target_1', sum),
                    sum_target_2_by_5_mon = ('target_2', sum),
                    sum_target_3_by_5_mon = ('target_3', sum),
                    sum_target_4_by_5_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    # Количество покупок продуктов за 180 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=6)].groupby(by='client_id').agg(
                    sum_target_1_by_6_mon = ('target_1', sum),
                    sum_target_2_by_6_mon = ('target_2', sum),
                    sum_target_3_by_6_mon = ('target_3', sum),
                    sum_target_4_by_6_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    
    # Период неактивности    
    period_noactive_target = select_mon_full_df[select_mon_full_df['target_1'] == 0].sort_values(by=['client_id', 'mon'])
    period_noactive_target['shift_mon'] = period_noactive_target.groupby('client_id')['mon'].shift(1)
    period_noactive_target['period_noactive_target'] = (period_noactive_target['mon'] - period_noactive_target['shift_mon']).dt.days.fillna(0)
    period_noactive_target = period_noactive_target.groupby(by='client_id').agg(
           max_period_noactive_target = ('period_noactive_target', max),
           min_period_noactive_target = ('period_noactive_target', min),
           avg_period_noactive_target = ('period_noactive_target', np.mean),
           median_period_noactive_target = ('period_noactive_target', np.median),
    )
    client_agg_df = client_agg_df.merge(period_noactive_target, left_index=True, right_index=True, how='left')
    
    union_client_agg_df = pd.concat([union_client_agg_df, client_agg_df])
    start_date = start_date + relativedelta(months=1)
union_client_agg_df.shape

In [ ]:
%%time
# Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
def series_to_int(col_df:pd.Series):
    """
    Перевод в целочисленные типы
    """
    min_val = col_df.min()
    max_val = col_df.max()
    if min_val >= -128 and max_val <= 127:
        col_df = col_df.astype('int8')
    elif min_val >= -32768 and max_val <= 32767:
        col_df = col_df.astype('int16')
    elif min_val >= -2147483648 and max_val <= 2147483647:
        col_df = col_df.astype('int32')
    else:
        col_df = col_df.astype('int64')
    return col_df

def compression_df(df:pd.DataFrame(), datetime_cols:List[str]=[], category_cols:List[str]=[]):
    """
    Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
    """
    float64_cols = list(df.select_dtypes(include='float64'))  
    df[float64_cols] = df[float64_cols].astype('float32')
    for col in df.columns:
        if col in category_cols:
            df[col] = df[col].astype('category')
        elif col in datetime_cols:
            if df[col].dtypes == 'object':
                df[col] = pd.to_datetime(df[col])
        # Если колонка содержит числа 
        elif is_integer_dtype(df[col]):
            if df[col].dtypes == 'int8':
                continue
            else:
                df[col] = series_to_int(df[col])
        elif is_float_dtype(df[col]):
            # Возможно ли перевести в число
            if np.array_equal(df[col].fillna(0), df[col].fillna(0).astype(int)):
                df[col] = df[col].fillna(0)
                df[col] = series_to_int(df[col])
    return df
union_client_agg_df = compression_df(union_client_agg_df, 
                            datetime_cols=['report_end' ,'report_next_end'],
                           )

mon_targets_df = compression_df(mon_targets_df, 
                            datetime_cols=['mon', 'pre_mon', 'next_mon'],
                           )
union_client_agg_df.shape, mon_targets_df.shape

## Объединяем с агрегированными данными по месяцам

In [ ]:
# union_client_agg_df['report_next_end'].min()
union_client_agg_df = union_client_agg_df.reset_index('client_id').set_index('report_next_end')
union_client_agg_df.shape

In [ ]:
%%time
union_client_agg_df = union_client_agg_df.merge(
                mon_targets_df.drop(columns=['mon', 'next_mon']).set_index('pre_mon').add_prefix('agg_premon_'),
                left_index=True,
                right_index=True,    
                how='left')
gc.collect()

union_client_agg_df = union_client_agg_df.merge(
                mon_targets_df.drop(columns=['pre_mon', 'next_mon']).set_index('mon').add_prefix('agg_curmon_'),
                left_index=True,
                right_index=True,    
                how='left')

gc.collect()

union_client_agg_df = union_client_agg_df.merge(
                mon_targets_df.drop(columns=['pre_mon', 'mon']).set_index('next_mon').add_prefix('agg_nxtmon_'),
                left_index=True,
                right_index=True,    
                how='left')

gc.collect()
union_client_agg_df = union_client_agg_df.fillna(0)
# union_client_agg_df = union_client_agg_df.reset_index().rename(columns={'index': 'report_next_end'}).set_index(['client_id','report_next_end'])
union_client_agg_df = union_client_agg_df.reset_index().rename(columns={'index': 'report_next_end'})
union_client_agg_df = union_client_agg_df.sort_values(by=['client_id','report_next_end']).set_index(['client_id','report_next_end'])

union_client_agg_df.shape

In [ ]:
%%time
# Сохраняем в файл оптимизированный файл 
union_client_agg_df.to_parquet(PATH_DATASET_OUTPUT + 'client_agg_target_09_06_2024.parquet')